In [1]:
import porespy as ps
import numpy as np

from matplotlib.animation import FuncAnimation, PillowWriter
from skimage import measure
from scipy.ndimage import distance_transform_edt
from scipy.signal import find_peaks
import numpy as np
from matplotlib import ticker
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
from skimage import measure, morphology
from scipy import ndimage
from scipy.ndimage import distance_transform_edt, label
import time
from matplotlib.animation import FuncAnimation, PillowWriter
import os
from scipy.stats import lognorm
from scipy.signal import find_peaks, savgol_filter
from scipy.interpolate import interp1d
import warnings
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib import ticker
from scipy.signal import savgol_filter, find_peaks
from scipy import interpolate
from scipy.ndimage import distance_transform_edt
from skimage import measure
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

warnings.filterwarnings('ignore')

# Configurar estilo de visualización
try:
    ps.visualization.set_mpl_style()
except:
    plt.style.use('default')

# Medio Poroso

In [2]:
# ========== FUNCIONES DE GENERACIÓN DE MEDIO POROSO ==========

def create_heterogeneous_tight_rock(shape, porosity_target=0.10, voxel_size_um=0.05, seed=None):
    """
    Genera un medio poroso heterogéneo optimizado para rocas tight.
    
    Características:
    - Distribución jerárquica de tamaños para garantizar heterogeneidad
    - Canales base para conectividad
    - Tamaños apropiados para la resolución
    """
    
    print(f"="*60)
    print(f"GENERANDO MEDIO POROSO HETEROGÉNEO OPTIMIZADO")
    print(f"="*60)
    print(f"Dimensiones: {shape}")
    print(f"Porosidad objetivo: {porosity_target:.1%}")
    print(f"Tamaño de voxel: {voxel_size_um:.3f} μm")
    
    if seed is not None:
        np.random.seed(seed)
    
    # PASO 1: Definir distribución jerárquica de tamaños
    # Asegurar heterogeneidad con 3 escalas distintas
    
    # Escala 1: Poros grandes (10% del total)
    large_pore_size_um = 0.8  # 16 voxels de radio
    n_large_pores = max(5, int(porosity_target * 0.1 * np.prod(shape) / 1000))
    
    # Escala 2: Poros medianos (30% del total)
    medium_pore_size_um = 0.4  # 8 voxels de radio
    n_medium_pores = max(20, int(porosity_target * 0.3 * np.prod(shape) / 500))
    
    # Escala 3: Poros pequeños (60% del total)
    small_pore_size_um = 0.2  # 4 voxels de radio
    n_small_pores = max(100, int(porosity_target * 0.6 * np.prod(shape) / 200))
    
    print(f"\nDistribución jerárquica de poros:")
    print(f"• Grandes ({large_pore_size_um:.1f} μm): {n_large_pores} poros")
    print(f"• Medianos ({medium_pore_size_um:.1f} μm): {n_medium_pores} poros")
    print(f"• Pequeños ({small_pore_size_um:.1f} μm): {n_small_pores} poros")
    
    # Inicializar medio
    medium = np.zeros(shape, dtype=bool)
    
    # PASO 2: Crear red de canales base para conectividad
    print(f"\nCreando red de canales base...")
    
    # Canales principales en las 3 direcciones
    channel_radius = int(small_pore_size_um / (2 * voxel_size_um))
    
    # Canal X
    y_center, z_center = shape[1]//2, shape[0]//2
    for x in range(shape[2]):
        for y in range(max(0, y_center-channel_radius), min(shape[1], y_center+channel_radius+1)):
            for z in range(max(0, z_center-channel_radius), min(shape[0], z_center+channel_radius+1)):
                if (y-y_center)**2 + (z-z_center)**2 <= channel_radius**2:
                    medium[z, y, x] = True
    
    # Canal Y
    x_center, z_center = shape[2]//2, shape[0]//2
    for y in range(shape[1]):
        for x in range(max(0, x_center-channel_radius), min(shape[2], x_center+channel_radius+1)):
            for z in range(max(0, z_center-channel_radius), min(shape[0], z_center+channel_radius+1)):
                if (x-x_center)**2 + (z-z_center)**2 <= channel_radius**2:
                    medium[z, y, x] = True
    
    # Canal Z
    x_center, y_center = shape[2]//2, shape[1]//2
    for z in range(shape[0]):
        for x in range(max(0, x_center-channel_radius), min(shape[2], x_center+channel_radius+1)):
            for y in range(max(0, y_center-channel_radius), min(shape[1], y_center+channel_radius+1)):
                if (x-x_center)**2 + (y-y_center)**2 <= channel_radius**2:
                    medium[z, y, x] = True
    
    # Canales diagonales para mejor conectividad
    for i in range(0, shape[0], shape[0]//4):
        for j in range(0, shape[1], shape[1]//4):
            start_x = 0 if (i+j)%2 == 0 else shape[2]-1
            end_x = shape[2]-1 if start_x == 0 else 0
            
            # Crear canal diagonal
            for t in np.linspace(0, 1, shape[2]):
                x = int(start_x + t * (end_x - start_x))
                y = int(j + t * shape[1]//4)
                z = int(i + t * shape[0]//4)
                
                if 0 <= y < shape[1] and 0 <= z < shape[0]:
                    for dy in range(-channel_radius, channel_radius+1):
                        for dz in range(-channel_radius, channel_radius+1):
                            if dy**2 + dz**2 <= channel_radius**2:
                                yy, zz = y+dy, z+dz
                                if 0 <= yy < shape[1] and 0 <= zz < shape[0]:
                                    medium[zz, yy, x] = True
    
    base_porosity = np.sum(medium) / medium.size
    print(f"Porosidad base de canales: {base_porosity:.3f}")
    
    # PASO 3: Colocar poros jerárquicamente
    print(f"\nColocando poros jerárquicamente...")
    
    # Coordenadas para esferas
    z_coords, y_coords, x_coords = np.mgrid[0:shape[0], 0:shape[1], 0:shape[2]]
    
    # Función auxiliar para colocar esferas
    def place_sphere(center, radius_um):
        radius_vox = radius_um / (2 * voxel_size_um)
        cz, cy, cx = center
        
        # Aplicar variación para heterogeneidad
        aspect_ratio = np.random.uniform(0.7, 1.3, size=3)
        
        sphere_mask = (
            ((z_coords - cz) / (radius_vox * aspect_ratio[0]))**2 + 
            ((y_coords - cy) / (radius_vox * aspect_ratio[1]))**2 + 
            ((x_coords - cx) / (radius_vox * aspect_ratio[2]))**2
        ) <= 1.0
        
        return sphere_mask
    
    # Colocar poros grandes
    print(f"  Colocando {n_large_pores} poros grandes...")
    margin_large = int(large_pore_size_um / (2 * voxel_size_um)) + 2
    
    for i in range(n_large_pores):
        # Distribución espacial más uniforme
        sector_x = (i % 3) * shape[2] // 3
        sector_y = ((i // 3) % 3) * shape[1] // 3
        sector_z = ((i // 9) % 3) * shape[0] // 3
        
        center = (
            np.random.randint(margin_large + sector_z, min(shape[0]-margin_large, sector_z + shape[0]//3)),
            np.random.randint(margin_large + sector_y, min(shape[1]-margin_large, sector_y + shape[1]//3)),
            np.random.randint(margin_large + sector_x, min(shape[2]-margin_large, sector_x + shape[2]//3))
        )
        
        # Variar tamaño para heterogeneidad
        size_variation = np.random.uniform(0.8, 1.2)
        medium |= place_sphere(center, large_pore_size_um * size_variation)
    
    # Colocar poros medianos
    print(f"  Colocando {n_medium_pores} poros medianos...")
    margin_medium = int(medium_pore_size_um / (2 * voxel_size_um)) + 1
    
    for i in range(n_medium_pores):
        for attempt in range(5):  # Múltiples intentos para buena distribución
            center = (
                np.random.randint(margin_medium, shape[0]-margin_medium),
                np.random.randint(margin_medium, shape[1]-margin_medium),
                np.random.randint(margin_medium, shape[2]-margin_medium)
            )
            
            # Preferir cerca de estructuras existentes
            if attempt > 2 or medium[center]:
                size_variation = np.random.uniform(0.7, 1.3)
                medium |= place_sphere(center, medium_pore_size_um * size_variation)
                break
    
    # Colocar poros pequeños
    print(f"  Colocando {n_small_pores} poros pequeños...")
    margin_small = int(small_pore_size_um / (2 * voxel_size_um)) + 1
    
    # Usar distance transform para colocar poros pequeños cerca de estructuras existentes
    dt = distance_transform_edt(~medium)
    
    for i in range(n_small_pores):
        # Buscar lugares cerca de poros existentes
        candidates = np.where((dt > 0) & (dt < 5))
        
        if len(candidates[0]) > 0:
            idx = np.random.randint(len(candidates[0]))
            center = (candidates[0][idx], candidates[1][idx], candidates[2][idx])
            
            # Verificar márgenes
            if (margin_small <= center[0] < shape[0]-margin_small and
                margin_small <= center[1] < shape[1]-margin_small and
                margin_small <= center[2] < shape[2]-margin_small):
                
                size_variation = np.random.uniform(0.6, 1.4)
                medium |= place_sphere(center, small_pore_size_um * size_variation)
    
    # PASO 4: Ajuste final de porosidad
    current_porosity = np.sum(medium) / medium.size
    print(f"\nPorosidad después de colocación: {current_porosity:.3f}")
    
    if abs(current_porosity - porosity_target) > 0.01:
        print(f"Ajustando porosidad...")
        
        if current_porosity > porosity_target:
            # Erosión suave
            iterations = 1 if current_porosity < porosity_target * 1.2 else 2
            struct = ndimage.generate_binary_structure(3, 1)
            medium = ndimage.binary_erosion(medium, structure=struct, iterations=iterations)
        elif current_porosity < porosity_target * 0.9:
            # Dilatación suave
            struct = ndimage.generate_binary_structure(3, 1)
            medium = ndimage.binary_dilation(medium, structure=struct, iterations=1)
    
    final_porosity = np.sum(medium) / medium.size
    
    # Verificar conectividad
    inlet = np.zeros(shape, dtype=bool)
    inlet[:, :, :5] = True
    connected = ps.filters.trim_disconnected_blobs(im=medium, inlets=inlet)
    connectivity = np.sum(connected) / np.sum(medium) if np.sum(medium) > 0 else 0
    
    print(f"\nResultados finales:")
    print(f"• Porosidad final: {final_porosity:.3f}")
    print(f"• Conectividad: {connectivity:.3f}")
    
    # Calcular heterogeneidad real
    dt_final = distance_transform_edt(medium)
    pore_radii = dt_final[medium] * voxel_size_um
    heterogeneity = np.std(pore_radii) / np.mean(pore_radii) if len(pore_radii) > 0 else 0
    print(f"• Coeficiente de heterogeneidad: {heterogeneity:.3f}")
    
    domain_size_um = [dim * voxel_size_um for dim in shape]
    
    return {
        'medium': medium,
        'shape': shape,
        'voxel_size_um': voxel_size_um,
        'domain_size_um': domain_size_um,
        'porosity_target': porosity_target,
        'porosity_actual': final_porosity,
        'connectivity': connectivity,
        'heterogeneity': heterogeneity,
        'pore_scales': {
            'large': large_pore_size_um,
            'medium': medium_pore_size_um,
            'small': small_pore_size_um
        }
    }


# Funciones Queu-Based Invasion Percolation

# Análisis QBIP


In [3]:

def run_invasion_fast(medium_info, inlet_face='z_min', max_iter=8000, 
                     subsample_analysis=True, inlet_thickness=3):
    """
    Versión optimizada de invasión con soporte para múltiples direcciones.
    
    Parameters
    ----------
    medium_info : dict
        Diccionario con información del medio poroso
    inlet_face : str
        Cara de entrada para la invasión. Opciones:
        - 'x_min': entrada por x=0 (antes 'left')
        - 'x_max': entrada por x=max
        - 'y_min': entrada por y=0
        - 'y_max': entrada por y=max
        - 'z_min': entrada por z=0 (arriba)
        - 'z_max': entrada por z=max (abajo)
        - 'omnidirectional': invasión desde todas las caras
        - 'corners': invasión desde las 8 esquinas
        - 'edges': invasión desde las 12 aristas
    max_iter : int
        Número máximo de iteraciones para QBIP
    subsample_analysis : bool
        Si hacer análisis con submuestreo
    inlet_thickness : int
        Grosor de la capa de entrada en voxels
    
    Returns
    -------
    dict
        Resultados de la simulación de invasión
    """
    
    print(f"\n{'='*60}")
    print(f"SIMULACIÓN DE INVASIÓN RÁPIDA")
    print(f"{'='*60}")
    print(f"Dirección de invasión: {inlet_face}")
    
    start_time = time.time()
    
    medium = medium_info['medium']
    shape = medium.shape
    
    # Generar inlet según la cara especificada
    inlet = np.zeros(shape, dtype=bool)
    
    # Diccionario de mapeo para compatibilidad con código anterior
    face_mapping = {
        'left': 'z_min',    # Compatibilidad con versión anterior
        'right': 'z_max',
        'top': 'y_min',
        'bottom': 'y_max',
        'front': 'x_min',
        'back': 'x_max'
    }
    
    # Convertir nombres antiguos si es necesario
    if inlet_face in face_mapping:
        inlet_face = face_mapping[inlet_face]
        print(f"  Convertido a nomenclatura nueva: {inlet_face}")
    
    # Crear inlet según la dirección especificada
    if inlet_face == 'x_min':
        inlet[:inlet_thickness, :, :] = True
        print(f"  Entrada por cara X mínima (x < {inlet_thickness})")
        
    elif inlet_face == 'x_max':
        inlet[-inlet_thickness:, :, :] = True
        print(f"  Entrada por cara X máxima (x > {shape[0]-inlet_thickness})")
        
    elif inlet_face == 'y_min':
        inlet[:, :inlet_thickness, :] = True
        print(f"  Entrada por cara Y mínima (y < {inlet_thickness})")
        
    elif inlet_face == 'y_max':
        inlet[:, -inlet_thickness:, :] = True
        print(f"  Entrada por cara Y máxima (y > {shape[1]-inlet_thickness})")
        
    elif inlet_face == 'z_min':
        inlet[:, :, :inlet_thickness] = True
        print(f"  Entrada por cara Z mínima (z < {inlet_thickness})")
        
    elif inlet_face == 'z_max':
        inlet[:, :, -inlet_thickness:] = True
        print(f"  Entrada por cara Z máxima (z > {shape[2]-inlet_thickness})")
        
    elif inlet_face == 'omnidirectional':
        # Invasión desde todas las caras exteriores
        print(f"  Entrada omnidireccional (todas las caras)")
        inlet[:inlet_thickness, :, :] = True  # x_min
        inlet[-inlet_thickness:, :, :] = True  # x_max
        inlet[:, :inlet_thickness, :] = True  # y_min
        inlet[:, -inlet_thickness:, :] = True  # y_max
        inlet[:, :, :inlet_thickness] = True  # z_min
        inlet[:, :, -inlet_thickness:] = True  # z_max
        
    elif inlet_face == 'corners':
        # Invasión desde las 8 esquinas
        print(f"  Entrada desde las 8 esquinas del dominio")
        corner_size = inlet_thickness * 2
        # Esquina 1: (0,0,0)
        inlet[:corner_size, :corner_size, :corner_size] = True
        # Esquina 2: (max,0,0)
        inlet[-corner_size:, :corner_size, :corner_size] = True
        # Esquina 3: (0,max,0)
        inlet[:corner_size, -corner_size:, :corner_size] = True
        # Esquina 4: (max,max,0)
        inlet[-corner_size:, -corner_size:, :corner_size] = True
        # Esquina 5: (0,0,max)
        inlet[:corner_size, :corner_size, -corner_size:] = True
        # Esquina 6: (max,0,max)
        inlet[-corner_size:, :corner_size, -corner_size:] = True
        # Esquina 7: (0,max,max)
        inlet[:corner_size, -corner_size:, -corner_size:] = True
        # Esquina 8: (max,max,max)
        inlet[-corner_size:, -corner_size:, -corner_size:] = True
        
    elif inlet_face == 'edges':
        # Invasión desde las 12 aristas
        print(f"  Entrada desde las 12 aristas del dominio")
        edge_thickness = inlet_thickness
        
        # Aristas paralelas al eje X (4 aristas)
        inlet[:, :edge_thickness, :edge_thickness] = True
        inlet[:, -edge_thickness:, :edge_thickness] = True
        inlet[:, :edge_thickness, -edge_thickness:] = True
        inlet[:, -edge_thickness:, -edge_thickness:] = True
        
        # Aristas paralelas al eje Y (4 aristas)
        inlet[:edge_thickness, :, :edge_thickness] = True
        inlet[-edge_thickness:, :, :edge_thickness] = True
        inlet[:edge_thickness, :, -edge_thickness:] = True
        inlet[-edge_thickness:, :, -edge_thickness:] = True
        
        # Aristas paralelas al eje Z (4 aristas)
        inlet[:edge_thickness, :edge_thickness, :] = True
        inlet[-edge_thickness:, :edge_thickness, :] = True
        inlet[:edge_thickness, -edge_thickness:, :] = True
        inlet[-edge_thickness:, -edge_thickness:, :] = True
        
    else:
        raise ValueError(f"inlet_face '{inlet_face}' no reconocido. "
                        f"Opciones válidas: x_min, x_max, y_min, y_max, z_min, z_max, "
                        f"omnidirectional, corners, edges")
    
    # Verificar conexión con el medio poroso
    inlet_overlap = np.sum(inlet & medium)
    total_inlet_voxels = np.sum(inlet)
    print(f"\nEstadísticas del inlet:")
    print(f"  Voxels totales del inlet: {total_inlet_voxels}")
    print(f"  Voxels de inlet conectados al medio: {inlet_overlap}")
    print(f"  Porcentaje de conexión: {100*inlet_overlap/total_inlet_voxels:.1f}%")
    
    # Si no hay suficiente conexión, dilatar el inlet
    if inlet_overlap < 10:
        print(f"  ⚠ Conexión insuficiente. Dilatando inlet...")
        inlet = ndimage.binary_dilation(inlet, iterations=2)
        inlet_overlap = np.sum(inlet & medium)
        print(f"  Después de expansión: {inlet_overlap} voxels conectados")
    
    # Eliminar regiones desconectadas del inlet
    print(f"\nEliminando regiones desconectadas...")
    medium_connected = ps.filters.trim_disconnected_blobs(im=medium, inlets=inlet)
    
    # Calcular estadísticas de conectividad
    connected_porosity = np.sum(medium_connected) / medium_connected.size
    original_porosity = np.sum(medium) / medium.size
    connectivity_ratio = np.sum(medium_connected) / np.sum(medium) if np.sum(medium) > 0 else 0
    
    print(f"\nEstadísticas de conectividad:")
    print(f"  Porosidad original: {original_porosity:.3f}")
    print(f"  Porosidad conectada: {connected_porosity:.3f}")
    print(f"  Ratio de conectividad: {connectivity_ratio:.3f}")
    
    # Ejecutar simulación QBIP
    print(f"\nEjecutando QBIP (máx {max_iter} iteraciones)...")
    print(f"  Método: Invasion percolation con trapping")
    
    try:
        inv = ps.simulations.ibip(
            im=medium_connected,
            inlets=inlet,
            maxiter=max_iter
        )
        
        inv_seq = inv.inv_sequence
        inv_satn = ps.filters.seq_to_satn(seq=inv_seq, im=medium_connected)
        
    except Exception as e:
        print(f"  ⚠ Error en QBIP: {e}")
        print(f"  Usando valores por defecto...")
        inv_seq = np.zeros_like(medium_connected, dtype=int)
        inv_satn = np.zeros_like(medium_connected, dtype=float)
    
    # Calcular métricas de invasión
    max_satn = np.max(inv_satn)
    invaded_voxels = np.sum(inv_satn > 0)
    total_pore_voxels = np.sum(medium_connected)
    efficiency = invaded_voxels / total_pore_voxels if total_pore_voxels > 0 else 0
    
    # Análisis por dirección para modos especiales
    if inlet_face in ['omnidirectional', 'corners', 'edges']:
        print(f"\nAnálisis de penetración por dirección:")
        
        # Calcular profundidad de invasión en cada dirección
        invaded_region = inv_satn > 0
        if np.any(invaded_region):
            invaded_coords = np.where(invaded_region)
            
            x_penetration = (np.max(invaded_coords[0]) - np.min(invaded_coords[0])) / shape[0]
            y_penetration = (np.max(invaded_coords[1]) - np.min(invaded_coords[1])) / shape[1]
            z_penetration = (np.max(invaded_coords[2]) - np.min(invaded_coords[2])) / shape[2]
            
            print(f"  Penetración en X: {x_penetration:.3f}")
            print(f"  Penetración en Y: {y_penetration:.3f}")
            print(f"  Penetración en Z: {z_penetration:.3f}")
            
            # Centro de masa de la región invadida
            com_x = np.mean(invaded_coords[0]) / shape[0]
            com_y = np.mean(invaded_coords[1]) / shape[1]
            com_z = np.mean(invaded_coords[2]) / shape[2]
            print(f"  Centro de masa (normalizado): ({com_x:.2f}, {com_y:.2f}, {com_z:.2f})")
    
    elapsed = time.time() - start_time
    
    print(f"\n{'='*60}")
    print(f"RESUMEN DE RESULTADOS")
    print(f"{'='*60}")
    print(f"Tiempo de simulación: {elapsed:.1f}s")
    print(f"Saturación máxima alcanzada: {max_satn:.3f}")
    print(f"Eficiencia de invasión: {efficiency:.3f}")
    print(f"Voxels invadidos: {invaded_voxels:,} / {total_pore_voxels:,}")
    
    # Preparar resultados
    results = {
        'medium_original': medium,
        'medium_connected': medium_connected,
        'inlets': inlet,
        'inv_sequence': inv_seq,
        'inv_saturation': inv_satn,
        'inlet_face': inlet_face,
        'inlet_thickness': inlet_thickness,
        'max_saturation': max_satn,
        'invasion_efficiency': efficiency,
        'connectivity_ratio': connectivity_ratio,
        'invaded_voxels': invaded_voxels,
        'total_pore_voxels': total_pore_voxels,
        'medium_info': medium_info,
        'simulation_time': elapsed
    }
    
    # Añadir análisis direccional si aplica
    if inlet_face in ['omnidirectional', 'corners', 'edges'] and np.any(inv_satn > 0):
        results['directional_analysis'] = {
            'x_penetration': x_penetration,
            'y_penetration': y_penetration,
            'z_penetration': z_penetration,
            'center_of_mass': (com_x, com_y, com_z)
        }
    
    return results

# ========== FUNCIONES DE ANÁLISIS DE PRESIÓN CAPILAR ==========

def analyze_capillary_pressure_fast(invasion_results):
    """
    Análisis rápido de curva PC y radio crítico.
    """
    
    print(f"\n{'='*60}")
    print(f"ANÁLISIS DE PRESIÓN CAPILAR")
    print(f"{'='*60}")
    
    inv_seq = invasion_results['inv_sequence']
    medium = invasion_results['medium_connected']
    voxel_size = invasion_results['medium_info']['voxel_size_um']
    
    # Parámetros de Washburn
    surface_tension = 0.078  # N/m
    contact_angle = 180  # grados
    cos_theta = np.cos(np.radians(contact_angle))
    
    # Distance transform
    dt = distance_transform_edt(medium)
    
    # Submuestrear secuencia para velocidad
    unique_seq = np.unique(inv_seq[inv_seq > 0])
    step = max(1, len(unique_seq) // 200)  # Máximo 200 puntos
    sampled_seq = unique_seq[::step]
    
    print(f"Analizando {len(sampled_seq)} puntos de {len(unique_seq)} totales")
    
    saturations = []
    pressures = []
    radii = []
    
    for seq_val in sampled_seq:
        mask = (inv_seq <= seq_val) & (inv_seq > 0)
        saturation = np.sum(mask) / np.sum(medium)
        
        new_invaded = inv_seq == seq_val
        if np.any(new_invaded):
            radius_vox = np.max(dt[new_invaded])
            radius_um = radius_vox * voxel_size
        else:
            radius_um = 0
        
        if radius_um > 0:
            pressure_pa = -2 * surface_tension * cos_theta / (radius_um * 1e-6)
            pressure_psi = pressure_pa * 1.45038e-4
        else:
            pressure_psi = 0
        
        saturations.append(saturation)
        pressures.append(pressure_psi)
        radii.append(radius_um)
    
    saturations = np.array(saturations)
    pressures = np.array(pressures)
    radii = np.array(radii)
    
    # Filtrar válidos
    valid = (pressures > 0) & (pressures < np.inf)
    saturations = saturations[valid]
    pressures = pressures[valid]
    radii = radii[valid]
    
    # Análisis de heterogeneidad
    if len(pressures) > 10:
        # Calcular envolvente (picos locales)
        peaks, _ = find_peaks(pressures, prominence=0.1*np.std(pressures))
        
        if len(peaks) > 2:
            print(f"Encontrados {len(peaks)} picos en la curva PC")
            # Interpolar envolvente
            envelope_sat = saturations[peaks]
            envelope_pres = pressures[peaks]
            
            # Ordenar por saturación
            sort_idx = np.argsort(envelope_sat)
            envelope_sat = envelope_sat[sort_idx]
            envelope_pres = envelope_pres[sort_idx]
            
            # Contar escalones significativos
            pressure_jumps = np.diff(np.log10(envelope_pres + 1))
            significant_jumps = np.sum(np.abs(pressure_jumps) > 0.1)
            
            print(f"Escalones significativos en envolvente: {significant_jumps}")
        else:
            print(f"⚠️ Solo {len(peaks)} picos - baja heterogeneidad")
            envelope_sat = saturations
            envelope_pres = pressures
    else:
        envelope_sat = saturations
        envelope_pres = pressures
    
    # Determinar radio crítico
    rc_inflection = None
    rc_percentile = None
    
    if len(radii) > 5:
        # Método 1: Punto de inflexión
        try:
            # Suavizar
            if len(saturations) > 11:
                sat_smooth = savgol_filter(saturations, 11, 3)
                log_p = np.log10(pressures + 1)
                dS_dlogP = np.gradient(sat_smooth, log_p)
                
                max_idx = np.argmax(dS_dlogP)
                rc_inflection = radii[max_idx]
                pc_inflection = pressures[max_idx]
                sc_inflection = saturations[max_idx]
            else:
                rc_inflection = None
        except:
            rc_inflection = None
        
        # Método 2: Percentil r35
        invaded_mask = inv_seq > 0
        invaded_radii = dt[invaded_mask] * voxel_size
        rc_percentile = np.percentile(invaded_radii, 35)
        
        print(f"\nRadios críticos:")
        if rc_inflection:
            print(f"• Inflexión: {rc_inflection:.3f} μm")
        print(f"• Percentil r35: {rc_percentile:.3f} μm")
        
        rc_final = rc_inflection if rc_inflection else rc_percentile
    else:
        rc_final = np.mean(radii) if len(radii) > 0 else 0
        rc_percentile = rc_final
    
    return {
        'saturations': saturations,
        'pressures': pressures,
        'radii': radii,
        'envelope_saturations': envelope_sat if 'envelope_sat' in locals() else saturations,
        'envelope_pressures': envelope_pres if 'envelope_pres' in locals() else pressures,
        'critical_radius': rc_final,
        'rc_inflection': rc_inflection,
        'rc_percentile': rc_percentile,
        'heterogeneity_indicator': len(peaks) if 'peaks' in locals() else 1
    }


In [4]:
def create_animated_invasion_pc_3d_improved(invasion_results, pc_analysis, fps=2, max_frames=30, save_static_frames=True):
    """
    Crea una animación GIF mejorada mostrando:
    - Izquierda: Vista isométrica 3D del progreso de la invasión
    - Derecha: Curva PC completa con marcador móvil y doble eje (Radio/Presión)
    
    Además genera frames estáticos en momentos clave:
    - Frame 1: Estado inicial (solo medio poroso)
    - Frame 2: Estado al 33% de saturación
    - Frame 3: Estado al 66% de saturación  
    - Frame 4: Estado final
    - Frame 5: Curva PC completa con formato MICP
    """
    # ============= CONFIGURACIÓN GLOBAL DE ESTILO =============
    # Definir tamaños de fuente consistentes
    FONT_SIZE_MAIN = 20        # Tamaño principal para ejes y etiquetas
    FONT_SIZE_TITLE = 22       # Tamaño para títulos de subplots
    FONT_SIZE_SUPTITLE = 24    # Tamaño para título principal
    FONT_SIZE_SMALL = 16       # Tamaño para anotaciones y textos secundarios
    FONT_SIZE_LEGEND = 14  
    
    
    # ============= CONFIGURACIÓN GLOBAL DE ESTILO =============
    #FONT_SIZE_MAIN = 16
    #FONT_SIZE_TITLE = 18
    #FONT_SIZE_SUPTITLE = 20
    #FONT_SIZE_SMALL = 14
    #FONT_SIZE_LEGEND = 12
    
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.serif': [ 'Sans-Serif' ],  #'Times New Roman', 'DejaVu Serif', 'Times', 'serif'],
        'font.size': FONT_SIZE_MAIN,
        'axes.labelsize': FONT_SIZE_MAIN,
        'axes.titlesize': FONT_SIZE_TITLE,
        'xtick.labelsize': FONT_SIZE_MAIN,
        'ytick.labelsize': FONT_SIZE_MAIN,
        'legend.fontsize': FONT_SIZE_LEGEND,
        'figure.titlesize': FONT_SIZE_SUPTITLE,
        'axes.linewidth': 1.5,
        'lines.linewidth': 2.5,
        'lines.markersize': 10,
        #'figure.figsize': [8, 7],  # Cambiar de (6,6) a (8,7)
        'figure.dpi': 300,
        'grid.alpha': 0.2,
        'grid.linestyle': '-'
    })
    
    print("\nCreando animación 3D mejorada de invasión y presión capilar...")
    
    # Extraer datos necesarios
    medium = invasion_results['medium_connected']
    inv_satn = invasion_results['inv_saturation']
    inv_seq = invasion_results['inv_sequence']
    voxel_size = invasion_results['medium_info']['voxel_size_um']
    
    # Obtener parámetros del dominio
    domain_size_um = invasion_results['medium_info'].get('domain_size_um', None)
    if domain_size_um is None:
        shape = invasion_results['medium_info'].get('shape', medium.shape)
        domain_size_um = [dim * voxel_size for dim in shape]
    
    print(f"Dominio para animación: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³")
    
    # Parámetros físicos
    surface_tension = 0.0785
    contact_angle = 180
    cos_theta = np.cos(np.radians(contact_angle))
    
    # Submuestrear para visualización 3D
    step = 3
    scale_factor = voxel_size * step
    medium_vis = medium[::step, ::step, ::step]
    inv_satn_vis = inv_satn[::step, ::step, ::step]
    inv_seq_vis = inv_seq[::step, ::step, ::step]
    
    # Pre-calcular isosuperficie del medio
    print("Pre-calculando isosuperficie del medio...")
    verts_medium, faces_medium, _, _ = measure.marching_cubes(medium_vis.astype(float), level=0.5)
    verts_medium_um = verts_medium * scale_factor
    
    # PRE-CALCULAR LA CURVA PC COMPLETA
    print("Calculando curva de presión capilar completa...")
    dt = distance_transform_edt(medium)
    
    saturations_full = []
    pressures_full = []
    radii_full = []
    sequence_values_full = []
    
    unique_sequences = np.unique(inv_seq[inv_seq > 0])
    step_seq = max(1, len(unique_sequences) // 500)
    seq_values = unique_sequences[::step_seq]
    
    for seq_val in seq_values:
        mask = (inv_seq <= seq_val) & (inv_seq > 0)
        sat = np.sum(mask) / np.sum(medium) * 100
        
        new_invaded = inv_seq == seq_val
        if np.any(new_invaded):
            radius_vox = np.max(dt[new_invaded])
            radius_um = radius_vox * voxel_size
            
            if radius_um > 0:
                pressure_pa = -2 * surface_tension * cos_theta / (radius_um * 1e-6)
                pressure_psi = pressure_pa * 1.45038e-4
                
                if 0 < pressure_psi < np.inf:
                    saturations_full.append(sat)
                    pressures_full.append(pressure_psi)
                    radii_full.append(radius_um)
                    sequence_values_full.append(seq_val)
    
    saturations_full = np.array(saturations_full)
    pressures_full = np.array(pressures_full)
    radii_full = np.array(radii_full)
    sequence_values_full = np.array(sequence_values_full)
    
    # Identificar picos en la curva
    peaks, properties = find_peaks(pressures_full, 
                                 prominence=0.5*np.std(pressures_full),
                                 distance=5)
    
    if len(peaks) > 20:
        prominences = properties.get('prominences', np.ones(len(peaks)))
        sorted_indices = np.argsort(prominences)[::-1]
        peaks = peaks[sorted_indices[:20]]
        peaks = np.sort(peaks)
    
        # Identificar picos en la curva
    peaks1, properties1 = find_peaks(pressures_full, 
                                 prominence=0.5*np.std(pressures_full),
                                 distance=1)
    
    
    # Preparar frames de animación
    unique_satns = np.unique(inv_satn[inv_satn > 0])
    if len(unique_satns) > max_frames:
        indices = np.linspace(0, len(unique_satns)-1, max_frames, dtype=int)
        frame_saturations = unique_satns[indices]
    else:
        frame_saturations = unique_satns
    
    # Configurar límites fijos para PC y radios
    if len(pressures_full) > 0:
        pres_min = np.percentile(pressures_full, 1)
        pres_max = np.percentile(pressures_full, 99)
        rad_min = np.percentile(radii_full, 1)
        rad_max = np.percentile(radii_full, 99)
    else:
        pres_min, pres_max = 10, 1000
        rad_min, rad_max = 0.1, 100
    
    # ========== FUNCIÓN PARA CREAR FRAMES 3D ESTÁTICOS (SOLO MEDIO POROSO) ==========
    def create_3d_frame_only(frame_idx, save_static=False, frame_name=""):
        """Crea frames mostrando solo la vista 3D del medio poroso siendo invadido"""
        
        fig = plt.figure(figsize=(12, 8), facecolor='white')
        ax = fig.add_subplot(111, projection='3d', position=[0.15, 0.1, 0.75, 0.85])
        
        ax.view_init(elev=30, azim=45)
        
        target_satn = frame_saturations[frame_idx] if frame_idx < len(frame_saturations) else frame_saturations[0]
        
        # Medio poroso semi-transparente
        ax.plot_trisurf(verts_medium_um[:, 0], verts_medium_um[:, 1], verts_medium_um[:, 2],
                       triangles=faces_medium, color='lightblue', alpha=0.2, 
                       edgecolor='none', linewidth=0)
        
        # Invasión (solo si no es el frame inicial)
        if frame_idx > 0:
            invasion_mask = (inv_satn_vis <= target_satn) & (inv_satn_vis > 0)
            
            if np.sum(invasion_mask) > 10:
                try:
                    verts_inv, faces_inv, _, _ = measure.marching_cubes(
                        invasion_mask.astype(float), level=0.5
                    )
                    verts_inv_um = verts_inv * scale_factor
                    
                    centroids = np.mean(verts_inv[faces_inv], axis=1)
                    colors = np.zeros((len(faces_inv), 3))
                    
                    # Colores grises metálicos
                    progress_colors = centroids[:, 0] / medium_vis.shape[2]
                    colors[:, 0] = 0.25 + 0.15 * progress_colors
                    colors[:, 1] = 0.25 + 0.15 * progress_colors
                    colors[:, 2] = 0.3 + 0.15 * progress_colors
                    
                    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
                    tri_collection = Poly3DCollection(verts_inv_um[faces_inv], 
                                                     facecolors=colors, 
                                                     alpha=0.9,
                                                     edgecolor='none',
                                                     linewidth=0)
                    ax.add_collection3d(tri_collection)
                except Exception as e:
                    pass
        
        # Configurar ejes con etiquetas visibles
        ax.set_xlim([0, domain_size_um[2]])
        ax.set_ylim([0, domain_size_um[1]])
        ax.set_zlim([0, domain_size_um[0]])
        
        ax.set_xlabel('X (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        ax.set_ylabel('Y (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        
        
            # CONFIGURACIÓN CORRECTA PARA EL EJE Z
        
        ax.set_zlabel('Z (μm)', fontsize=FONT_SIZE_MAIN, rotation=0, labelpad=15)
        
        ax.tick_params(axis='x', labelsize=FONT_SIZE_SMALL)
        ax.tick_params(axis='y', labelsize=FONT_SIZE_SMALL)
        ax.tick_params(axis='z', labelsize=FONT_SIZE_SMALL)
        
        ax.xaxis.set_major_locator(plt.MaxNLocator(5))
        ax.yaxis.set_major_locator(plt.MaxNLocator(5))
        ax.zaxis.set_major_locator(plt.MaxNLocator(5))
        
        # Título simple con saturación
        if frame_idx == 0:
            title = f'Saturación: 0.0%'
        else:
            title = f'Saturación: {target_satn * 100:.1f}%'
        ax.set_title(title, fontsize=FONT_SIZE_TITLE, pad=10)
        
        # Eliminar el panel gris del fondo
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.grid(True, alpha=0.3)
        
       # plt.tight_layout()
        
            # IMPORTANTE: Ajustar el subplot para dar más espacio
        plt.subplots_adjust(left=0.0, right=1.0, bottom=0.0, top=0.95)
    
    # Usar tight_layout con pad adicional
        #plt.tight_layout(pad=2)
        
        
        return fig
    
    # ========== GENERAR FRAMES 3D ESTÁTICOS ==========
    if save_static_frames:
        # Definir saturaciones para frames estáticos
        min_sat = frame_saturations[0]
        max_sat = frame_saturations[-1]
        
        static_configs = [
            (0, 'Initial'),  # Frame inicial sin invasión
            (int(len(frame_saturations) * 0.33), 'Intermediate_33%'),
            (int(len(frame_saturations) * 0.66), 'Intermediate_66%'),
            (len(frame_saturations) - 1, 'Final')
        ]
        
        print(f"\nGenerando frames 3D estáticos...")
        
        for i, (idx, name) in enumerate(static_configs):
            print(f"Generando frame estático {i+1}/4: {name}")
            fig = create_3d_frame_only(idx, save_static=True, frame_name=name)
            filename = f'invasion_3d_frame_{i+1}_{name.lower()}.png'
            fig.savefig(filename, dpi=300,bbox_inches='tight',  pad_inches=0.5,facecolor='white')
            plt.close(fig)
            print(f"  ✓ Guardado: {filename} (300 DPI)")
            
############################################################################################################################################################        
        # ========== FRAME 5: CURVA PC Invasion por volumen ==========
        print("Generando frame 5: Curva PC completa (formato MICP)")
            
        fig_pc = plt.figure(figsize=(8, 7), facecolor='white')
        ax = fig_pc.add_subplot(111)


        # Configurar grid
        ax.grid(True, which="both", ls="-", alpha=0.2)
        
        # Configurar eje X inferior (radio de poro) - INVERTIDO
        
    # Añadir margen del 20% en escala log
        log_margin = 0.2
        min_radius_plot = 10**(np.log10(rad_min) - log_margin)
        max_radius_plot = 10**(np.log10(rad_max) + log_margin)
    # P = -2γcos(θ)/r, donde r está en metros
    # Calcular rangos de presión basados en los rangos de radio
        gamma = 0.078  # N/m
        theta = 180    # grados
        theta_rad = np.radians(theta)
        pressure_min = np.abs(2 * gamma * np.cos(theta_rad) / (max_radius_plot * 1e-6)) * 0.000145038
        pressure_max = np.abs(2 * gamma * np.cos(theta_rad) / (min_radius_plot * 1e-6)) * 0.000145038
        
        ax.set_xscale('log')
        ax.set_xlim([max_radius_plot, min_radius_plot])  # Invertido: mayor a menor
        ax.set_xlabel('Radio de garganta de poro (μm)', fontsize=FONT_SIZE_MAIN,labelpad=10)
        #ax.tick_params(axis='x', labelsize=FONT_SIZE_SMALL)
        ax.tick_params(axis='x', which='major', labelsize=FONT_SIZE_MAIN, pad=10)
        ax.tick_params(axis='x', which='minor', labelsize=0)  # Oculta etiquetas menores

        
        # Configurar eje Y (saturación)
        ax.set_ylabel('Saturación (%)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        ax.set_ylim([-1, 101])
        ax.yaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(5))
        ax.tick_params(axis='y', which='major', length=6, width=1.2, color='black')

        #ax.tick_params(axis='y', labelsize=FONT_SIZE_SMALL)
        
        # Crear eje superior (Presión)
        ax2 = ax.twiny()
        ax2.set_xscale('log')
        ax2.set_xlim([pressure_min, pressure_max])
        ax2.set_xlabel('Presión (psi)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        
        ax2.tick_params(axis='x', which='major', labelsize=FONT_SIZE_MAIN, pad=10)
        ax2.tick_params(axis='x', which='minor', labelsize=0)  # Oculta etiquetas menores
        
        # ========== CALCULAR PSD (dS/dlog(r)) ==========
        # Ordenar datos por radio descendente
        sorted_indices = np.argsort(radii_full)[::-1]
        radii_sorted_desc = radii_full[sorted_indices]
        sats_sorted_desc = saturations_full[sorted_indices]
        
        # Calcular diferencias de saturación
        dS = np.diff(sats_sorted_desc)
        
        # Calcular el ancho logarítmico de cada intervalo
        log_bin_widths = []
        for i in range(len(radii_sorted_desc)-1):
            dlog_r = np.abs(np.log10(radii_sorted_desc[i]) - np.log10(radii_sorted_desc[i+1]))
            if dlog_r == 0:
                dlog_r = 0.001  # Evitar división por cero
            log_bin_widths.append(dlog_r)
        
        log_bin_widths = np.array(log_bin_widths)
        
        # Calcular dS/dlog(r)
        # Usar los valores absolutos ya que queremos la magnitud del cambio
        frequency_distribution = np.abs(dS) / log_bin_widths
        
        # Puntos medios para graficar
        radii_midpoints = []
        for i in range(len(radii_sorted_desc)-1):
            mid = np.sqrt(radii_sorted_desc[i] * radii_sorted_desc[i+1])  # Media geométrica
            radii_midpoints.append(mid)
        radii_midpoints = np.array(radii_midpoints)
        
        # Aplicar suavizado si hay suficientes puntos
        if len(frequency_distribution) > 5:
            from scipy.signal import savgol_filter
            window = min(5, len(frequency_distribution))
            if window % 2 == 0:
                window -= 1
            frequency_distribution_smooth = savgol_filter(frequency_distribution, 
                                                         window_length=window, 
                                                         polyorder=2)
            frequency_distribution_smooth = np.maximum(frequency_distribution_smooth, 0)
        else:
            frequency_distribution_smooth = frequency_distribution
        
        ax3 = ax.twinx()
        
        # Calcular el límite superior apropiado para la PSD
        if len(frequency_distribution_smooth) > 0:
            max_dist = np.max(frequency_distribution_smooth) * 1.2
        else:
            max_dist = 100  # Valor por defecto
        
        # Configurar el eje pero sin mostrar nada
        ax3.set_ylim([-1, max_dist])  
        
        ax3.set_ylabel('dS/dlog(r) (%/log(μm))', fontsize=FONT_SIZE_MAIN, alpha=0)
        ax3.tick_params(axis='y',labelsize=FONT_SIZE_MAIN, color='white')  # Hace blancos los números y ticks
        
        # Configurar los parámetros de los ticks
        ax3.tick_params(axis='y', which='major', labelsize=FONT_SIZE_MAIN, labelcolor='white', colors='white')
        ax3.tick_params(axis='y', which='minor', labelsize=FONT_SIZE_MAIN, labelcolor='white', colors='white')

        
       
        # Graficar la PSD en blanco (invisible pero presente para mantener proporción)
        ax3.semilogx(radii_midpoints, frequency_distribution_smooth, 
                     linestyle='--', linewidth=2.5, alpha=0)
 
                       
                       
        
        # Interpolar para curva más suave si hay pocos puntos
        if len(radii_full) > 3 and len(radii_full) < 50:
            from scipy import interpolate
            # Ordenar datos
            sorted_indices = np.argsort(radii_full)
            radii_sorted = radii_full[sorted_indices]
            sats_sorted = saturations_full[sorted_indices]
            
            # Crear interpolador
            f_interp = interpolate.interp1d(np.log10(radii_sorted), sats_sorted, 
                                           kind='linear', fill_value='extrapolate')
            
            # Generar más puntos
            radii_smooth = np.logspace(np.log10(rad_min), np.log10(rad_max), 200)
            sats_smooth = f_interp(np.log10(radii_smooth))
            sats_smooth = np.clip(sats_smooth, 0, 100)
            
            # Graficar curva suavizada
            ax.plot(radii_smooth, sats_smooth, 'b-', linewidth=2.5, alpha=0.8, label='$P_c$')
        else:
            # Graficar curva original
            ax.plot(radii_full, saturations_full, 'b-', linewidth=2.5, alpha=0.8, label='$P_c$')
        
        # Marcar picos/pore-throats si existen
        if len(peaks1) > 0:
            ax.plot(radii_full[peaks1], saturations_full[peaks1],'r^', markersize=8, label=f'Gargantas de poro', zorder=3)
                                                        #({len(peaks1)})
        # Marcar radio crítico si existe
        #if 'rc_inflection' in pc_analysis and not np.isnan(pc_analysis['rc_inflection']):
        #    idx_crit = np.argmin(np.abs(radii_full - pc_analysis['rc_inflection']))
        #    if idx_crit < len(radii_full):
        #        ax.plot(radii_full[idx_crit], saturations_full[idx_crit], 
        #               'go', markersize=10, label=f'$R_c$ = {pc_analysis["rc_inflection"]:.3f} μm', zorder=4)
        
        # Leyenda
        ax.legend(loc='upper left', fontsize=FONT_SIZE_LEGEND)
        
        # Título
        #ax.set_title('Capillary Pressure Curve', fontsize=FONT_SIZE_TITLE, pad=30)
        
        # Estadísticas en un cuadro
        #if len(saturations_full) > 0:
         #   stats_text = (f'Max saturation: {max(saturations_full):.1f}%\n'
          #               f'Pressure range: {min(pressures_full):.0f} - {max(pressures_full):.0f} psi\n'
           #              f'Radius range: {min(radii_full):.3f} - {max(radii_full):.3f} μm')
           # ax.text(0.95, 0.05, stats_text, transform=ax.transAxes,
            #       fontsize=FONT_SIZE_SMALL, ha='right', va='bottom',
             #      bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        plt.tight_layout()
        filename = 'capillary_pressure_curve_micp_format.png'
        fig_pc.savefig(filename, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close(fig_pc)
        print(f"  ✓ Guardado: {filename} (300 DPI)")
    
    # ========== CREAR ANIMACIÓN (mantiene el formato original) ==========
    print("\nGenerando animación...")
    
    # Crear figura para animación con ambos paneles
    fig_anim = plt.figure(figsize=(18, 9), facecolor='white')
    gs = fig_anim.add_gridspec(1, 2, width_ratios=[1, 1], wspace=0.15)
    ax1 = fig_anim.add_subplot(gs[0, 0], projection='3d')
    ax2 = fig_anim.add_subplot(gs[0, 1])
    
    def update(frame_idx):
        ax1.clear()
        ax2.clear()
        
        target_satn = frame_saturations[frame_idx]
        
        # Panel 1: Vista isométrica 3D
        ax1.plot_trisurf(verts_medium_um[:, 0], verts_medium_um[:, 1], verts_medium_um[:, 2],
                        triangles=faces_medium, color='lightblue', alpha=0.15, 
                        edgecolor='none', linewidth=0)
        
        # Invasión
        invasion_mask = (inv_satn_vis <= target_satn) & (inv_satn_vis > 0)
        
        if np.sum(invasion_mask) > 10:
            try:
                verts_inv, faces_inv, _, _ = measure.marching_cubes(invasion_mask.astype(float), level=0.5)
                verts_inv_um = verts_inv * scale_factor
                
                centroids = np.mean(verts_inv[faces_inv], axis=1)
                colors = np.zeros((len(faces_inv), 3))
                
                progress_colors = centroids[:, 0] / medium_vis.shape[2]
                colors[:, 0] = 0.25 + 0.15 * progress_colors
                colors[:, 1] = 0.25 + 0.15 * progress_colors
                colors[:, 2] = 0.3 + 0.15 * progress_colors
                
                from mpl_toolkits.mplot3d.art3d import Poly3DCollection
                tri_collection = Poly3DCollection(verts_inv_um[faces_inv], 
                                                 facecolors=colors, 
                                                 alpha=0.9,
                                                 edgecolor='none',
                                                 linewidth=0)
                ax1.add_collection3d(tri_collection)
            except Exception as e:
                pass
        
        # Configurar ejes 3D
        ax1.set_xlim([0, domain_size_um[2]])
        ax1.set_ylim([0, domain_size_um[1]])
        ax1.set_zlim([0, domain_size_um[0]])
        
        ax1.set_xlabel('X (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        ax1.set_ylabel('Y (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        ax1.set_zlabel('Z (μm)', fontsize=FONT_SIZE_MAIN, rotation=0, ha='right', labelpad=10)
        
        ax1.tick_params(axis='x', labelsize=FONT_SIZE_SMALL)
        ax1.tick_params(axis='y', labelsize=FONT_SIZE_SMALL)
        ax1.tick_params(axis='z', labelsize=FONT_SIZE_SMALL)
        
        ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
        ax1.yaxis.set_major_locator(plt.MaxNLocator(5))
        ax1.zaxis.set_major_locator(plt.MaxNLocator(5))
        
        ax1.set_title(f'\nSaturación: {target_satn * 100:.1f}%', fontsize=FONT_SIZE_TITLE)
        
        # Panel 2: Curva PC
        ax2.set_xscale('log')
        ax2.set_xlim([rad_max * 1.2, rad_min * 0.8])
        ax2.set_ylim([-1, 101])
        ax2.set_xlabel('Radio de garganta de poro (μm)', fontsize=FONT_SIZE_MAIN)
        ax2.tick_params(axis='x', labelcolor='black', labelsize=FONT_SIZE_MAIN)
        ax2.set_ylabel('Saturación (%)', fontsize=FONT_SIZE_MAIN)
        ax2.tick_params(axis='y', labelsize=FONT_SIZE_MAIN)
        ax2.grid(True, alpha=0.3, which='both')
        
        # Eje superior (Presión)
        ax3 = ax2.twiny()
        ax3.set_xscale('log')
        ax3.set_xlim([pres_min * 0.8, pres_max * 1.2])
        ax3.set_xlabel('Presión (psi)', fontsize=FONT_SIZE_MAIN)
        ax3.tick_params(axis='x', labelcolor='black', labelsize=FONT_SIZE_MAIN)
        
        ax2.set_title('Presión capilar', fontsize=FONT_SIZE_TITLE, pad=40)
        
        # Curva completa
        ax2.plot(radii_full, saturations_full, 'lightgray', linewidth=2, alpha=0.5, label='$P_c$')
        
        # Índice actual
        current_idx = np.argmin(np.abs(saturations_full - target_satn * 100))
        
        # Porción recorrida
        if current_idx > 0:
            ax2.plot(radii_full[:current_idx+1], saturations_full[:current_idx+1], 'g-', linewidth=3, alpha=0.8)
        
        # Marcador actual
        if current_idx < len(radii_full):
            current_radius = radii_full[current_idx]
            current_saturation = saturations_full[current_idx]
            
            ax2.plot(current_radius, current_saturation, 'ro', markersize=15, 
                    markerfacecolor='red', markeredgecolor='darkred', markeredgewidth=2, zorder=5)
            
            pulse_size = 20 + 5 * np.sin(frame_idx * 0.5)
            ax2.plot(current_radius, current_saturation, 'ro', markersize=pulse_size, 
                    fillstyle='none', markeredgecolor='red', markeredgewidth=1, alpha=0.5, zorder=4)
        
        # Marcar picos
        if len(peaks) > 0:
            ax2.plot(radii_full[peaks], saturations_full[peaks], 'b^', 
                    markersize=8, label='Gargantas de poro', zorder=3, alpha=0.6)
            
            passed_peaks = peaks[peaks <= current_idx]
            if len(passed_peaks) > 0:
                ax2.plot(radii_full[passed_peaks], saturations_full[passed_peaks], 
                        'b^', markersize=14, zorder=4)
        
        ax2.legend(loc='upper left', fontsize=FONT_SIZE_LEGEND)
        
        # Título general
        progress = (frame_idx / (len(frame_saturations)-1)) * 100 if len(frame_saturations) > 1 else 0
        domain_info = f" | Dominio: {domain_size_um[0]:.1f}×{domain_size_um[1]:.1f}×{domain_size_um[2]:.1f} μm³"
        fig_anim.suptitle(f'QBIP 3D Evolution - Progreso: {progress:.0f}%{domain_info}', 
                         fontsize=FONT_SIZE_SUPTITLE, y=0.99)
        
        ax1.view_init(elev=30, azim=45)
        plt.tight_layout(rect=[0, 0.03, 1, 1.6])
    
    # Crear animación
    from matplotlib.animation import FuncAnimation, PillowWriter
    anim = FuncAnimation(fig_anim, update, frames=len(frame_saturations), 
                        interval=1000//fps, repeat=True)
    
    filename = 'invasion_pc_3d_evolution_improved.gif'
    print(f"Guardando animación como {filename}...")
    anim.save(filename, writer=PillowWriter(fps=fps))
    print(f"  ✓ Animación guardada")
    
    plt.close(fig_anim)
    
    # Resumen de archivos generados
    print("\n" + "="*60)
    print("ARCHIVOS GENERADOS:")
    print("="*60)
    if save_static_frames:
        print("Frames 3D estáticos (300 DPI):")
        print("  1. invasion_3d_frame_1_initial.png (sin invasión)")
        print("  2. invasion_3d_frame_2_intermediate_33%.png")
        print("  3. invasion_3d_frame_3_intermediate_66%.png")
        print("  4. invasion_3d_frame_4_final.png")
        print("\nCurva de presión capilar (300 DPI):")
        print("  5. capillary_pressure_curve_micp_format.png")
    print("\nAnimación:")
    print(f"  • {filename}")
    print("="*60)
    
    return {
        'animation': anim,
        'radii_full': radii_full,
        'saturations_full': saturations_full,
        'pressures_full': pressures_full,
        'peaks1': peaks1,
        'sequence_values_full': sequence_values_full
    }

In [5]:
def analyze_jumps_by_pore_size_detailed(invasion_results, pc_analysis, 
                                       min_jump_size=0.0001,  # Umbral más bajo
                                       n_bins=20):

    """
    Análisis detallado de escalones por tamaño de garganta de poro
    """
    
    # Primero, obtener TODOS los escalones con umbral bajo
    print("\nAnalizando distribución detallada de escalones...")
    
    # Extraer datos necesarios
    medium = invasion_results['medium_connected']
    inv_seq = invasion_results['inv_sequence']
    voxel_size = invasion_results['medium_info']['voxel_size_um']
    
    # Parámetros físicos
    surface_tension = 0.078  # N/m
    contact_angle = 180  # grados
    cos_theta = np.cos(np.radians(contact_angle))
    
    dt = distance_transform_edt(medium)
    
    # Arrays para almacenar TODOS los eventos
    all_radii = []
    all_jump_sizes = []
    all_saturations = []
    all_pressures = []
    
    # Obtener secuencias únicas
    unique_seq = np.unique(inv_seq[inv_seq > 0])
    
    prev_saturation = 0
    prev_volume = 0
    
    # Analizar cada paso
    for i, seq in enumerate(unique_seq):
        if i % 1000 == 0:
            print(f"  Procesando paso {i}/{len(unique_seq)}...")
        
        current_mask = (inv_seq <= seq) & (inv_seq > 0)
        current_saturation = np.sum(current_mask) / np.sum(medium)
        current_volume = np.sum(current_mask)
        
        saturation_jump = current_saturation - prev_saturation
        
        # Registrar TODOS los saltos positivos
        if saturation_jump > min_jump_size:
            new_invaded = inv_seq == seq
            if np.any(new_invaded):
                radius_vox = np.max(dt[new_invaded])
                radius_um = radius_vox * voxel_size
                
                if radius_um > 0:
                    pressure_pa = -2 * surface_tension * cos_theta / (radius_um * 1e-6)
                    pressure_psi = pressure_pa * 1.45038e-4
                    
                    all_radii.append(radius_um)
                    all_jump_sizes.append(saturation_jump)
                    all_saturations.append(current_saturation)
                    all_pressures.append(pressure_psi)
        
        prev_saturation = current_saturation
        prev_volume = current_volume
    
    # Convertir a arrays
    all_radii = np.array(all_radii)
    all_jump_sizes = np.array(all_jump_sizes)
    all_saturations = np.array(all_saturations)
    all_pressures = np.array(all_pressures)
    
    print(f"Total de escalones detectados: {len(all_radii)}")
    
    # ANÁLISIS POR RANGOS DE TAMAÑO
    # Definir bins logarítmicos
    r_min, r_max = all_radii.min() * 0.9, all_radii.max() * 1.1
    bins = np.logspace(np.log10(r_min), np.log10(r_max), n_bins)
    
    # Estadísticas por bin
    bin_stats = []
    
    for i in range(len(bins)-1):
        mask = (all_radii >= bins[i]) & (all_radii < bins[i+1])
        
        if np.any(mask):
            stats = {
                'bin_center': np.sqrt(bins[i] * bins[i+1]),  # Media geométrica
                'bin_range': (bins[i], bins[i+1]),
                'count': np.sum(mask),
                'total_jump': np.sum(all_jump_sizes[mask]),
                'mean_jump': np.mean(all_jump_sizes[mask]),
                'std_jump': np.std(all_jump_sizes[mask]),
                'max_jump': np.max(all_jump_sizes[mask]),
                'radii': all_radii[mask],
                'jumps': all_jump_sizes[mask],
                'saturations': all_saturations[mask]
            }
        else:
            stats = {
                'bin_center': np.sqrt(bins[i] * bins[i+1]),
                'bin_range': (bins[i], bins[i+1]),
                'count': 0,
                'total_jump': 0,
                'mean_jump': 0,
                'std_jump': 0,
                'max_jump': 0,
                'radii': np.array([]),
                'jumps': np.array([]),
                'saturations': np.array([])
            }
        
        bin_stats.append(stats)
    
    return {
        'all_radii': all_radii,
        'all_jump_sizes': all_jump_sizes,
        'all_saturations': all_saturations,
        'all_pressures': all_pressures,
        'bin_stats': bin_stats,
        'bins': bins
    }


def plot_jump_size_distribution_detailed(jump_analysis_detailed, show_characteristic_radii=True):
    """
    Visualización de la distribución de escalones con formato MICP
    Similar al formato de las gráficas MICP estándar
    Con formato homogeneizado de fuentes.
    
    Args:
        jump_analysis_detailed: Diccionario con los datos del análisis de saltos
        show_characteristic_radii: Bool para mostrar o no los radios característicos
    """
    
    # ============= CONFIGURACIÓN GLOBAL DE ESTILO =============
    # Definir tamaños de fuente consistentes
    FONT_SIZE_MAIN = 20        # Tamaño principal para ejes y etiquetas
    FONT_SIZE_TITLE = 22       # Tamaño para títulos de subplots
    FONT_SIZE_SUPTITLE = 24    # Tamaño para título principal
    FONT_SIZE_SMALL = 16       # Tamaño para anotaciones y textos secundarios
    FONT_SIZE_LEGEND = 14      # Tamaño para leyendas
    
    # Configurar matplotlib para usar sans-serif
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.serif': ['Sans-Serif'], #'Times New Roman', 'DejaVu Serif', 'Times', 'serif'],
        'font.size': FONT_SIZE_MAIN,
        'axes.labelsize': FONT_SIZE_MAIN,
        'axes.titlesize': FONT_SIZE_TITLE,
        'xtick.labelsize': FONT_SIZE_MAIN,
        'ytick.labelsize': FONT_SIZE_MAIN,
        'legend.fontsize': FONT_SIZE_LEGEND,
   #     'axes.labelweight': 'bold',
   #     'axes.titleweight': 'bold',
        'figure.titlesize': FONT_SIZE_SUPTITLE,
   #     'figure.titleweight': 'bold',
        'axes.linewidth': 1.5,
        'lines.linewidth': 2.5,
        'lines.markersize': 10,
        'figure.figsize': [8, 7],
        'figure.dpi': 300,
        'grid.alpha': 0.2,
        'grid.linestyle': '-'
    })
    # =========================================================
    
    # Crear figura principal
    fig, ax1 = plt.subplots()
    
    # Eje derecho para distribución
    ax3 = ax1.twinx()
    
    # Eje superior para presión
    ax2 = ax1.twiny()
    
    # Preparar datos
    bin_stats = jump_analysis_detailed['bin_stats']
    bin_centers = np.array([stat['bin_center'] for stat in bin_stats])
    counts = np.array([stat['count'] for stat in bin_stats])
    total_jumps = np.array([stat['total_jump'] for stat in bin_stats])
    
    # Convertir radios de μm a nm para consistencia
    bin_centers_nm = bin_centers  # Ya están en μm según el código
    
    # Ordenar datos por radio descendente (similar a MICP)
    sorted_indices = np.argsort(bin_centers_nm)[::-1]
    bin_centers_nm_sorted = bin_centers_nm[sorted_indices]
    total_jumps_sorted = total_jumps[sorted_indices]
    
    # Calcular saturación acumulada correctamente (debe aumentar conforme disminuye el radio)
    cumulative_saturation = np.cumsum(total_jumps_sorted) * 100
    
    # Calcular distribución dS/dlog(r)
    # Primero, calcular los incrementos de saturación para cada bin
    dS = total_jumps_sorted * 100  # Cambio en saturación en %
    
    # Calcular el ancho logarítmico de cada bin
    log_bin_widths = []
    sorted_bin_centers = bin_centers[sorted_indices]  # en μm para el cálculo log
    
    for i in range(len(sorted_bin_centers)):
        if i < len(sorted_bin_centers) - 1:
            # Diferencia logarítmica entre bins consecutivos
            dlog_r = np.abs(np.log10(sorted_bin_centers[i]) - np.log10(sorted_bin_centers[i+1]))
        else:
            # Para el último bin, usar el ancho del anterior
            if i > 0:
                dlog_r = np.abs(np.log10(sorted_bin_centers[i-1]) - np.log10(sorted_bin_centers[i]))
            else:
                dlog_r = 0.1  # valor por defecto
        log_bin_widths.append(dlog_r)
    
    # Calcular dS/dlog(r)
    frequency_distribution = dS / np.array(log_bin_widths)
    
    # Aplicar suavizado si hay suficientes puntos
    if len(frequency_distribution) > 5:
        window = min(5, len(frequency_distribution))
        if window % 2 == 0:
            window -= 1
        frequency_distribution_smooth = savgol_filter(frequency_distribution, 
                                                     window_length=window, 
                                                     polyorder=2)
        frequency_distribution_smooth = np.maximum(frequency_distribution_smooth, 0)
    else:
        frequency_distribution_smooth = frequency_distribution
    
    # Configurar grid
    ax1.grid(True, which="both", ls="-", alpha=0.2)
    
    # Determinar rangos apropiados basados en los datos
    min_radius_data = np.min(bin_centers_nm_sorted)
    max_radius_data = np.max(bin_centers_nm_sorted)
    
    # Añadir margen del 20% en escala log
    log_margin = 0.2
    min_radius_plot = 10**(np.log10(min_radius_data) - log_margin)
    max_radius_plot = 10**(np.log10(max_radius_data) + log_margin)
    
    # Configurar eje X inferior (radio de poro)
    ax1.set_xscale('log')
    ax1.set_xlim(max_radius_plot, min_radius_plot)  # Invertido: mayor a menor
    ax1.tick_params(axis='both', which='major', labelsize=FONT_SIZE_MAIN, pad=10)
    ax1.tick_params(axis='both', which='minor', labelsize=FONT_SIZE_MAIN)
    ax1.set_xlabel('Radio de garganta de poro (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
    
    # Configurar eje Y izquierdo (saturación)
    ax1.set_ylabel('Saturación (%)', fontsize=FONT_SIZE_MAIN, )
    ax1.set_ylim(-1, 101)
    ax1.yaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(5))
    ax1.tick_params(axis='y', which='major', length=6, width=1.2, color='black')
    
    # Configurar eje X superior (presión)
    # Calcular presión usando la ecuación de Washburn
    gamma = 0.078  # N/m
    theta = 180    # grados
    theta_rad = np.radians(theta)
    
    
    # P = -2γcos(θ)/r, donde r está en metros
    # Calcular rangos de presión basados en los rangos de radio
    pressure_min = np.abs(2 * gamma * np.cos(theta_rad) / (max_radius_plot * 1e-6)) * 0.000145038
    pressure_max = np.abs(2 * gamma * np.cos(theta_rad) / (min_radius_plot * 1e-6)) * 0.000145038
    
    ax2.set_xscale('log')
    ax2.set_xlim(pressure_min, pressure_max)
    ax2.tick_params(axis='both', which='major', labelsize=FONT_SIZE_MAIN, pad=10)
    ax2.tick_params(axis='both', which='minor', labelsize=FONT_SIZE_MAIN)
    ax2.set_xlabel('Presión (psi)', fontsize=FONT_SIZE_MAIN,  labelpad=10)
    
    # Configurar eje Y derecho (distribución)
    ax3.set_ylabel('dS/dlog(r) (%/log(μm))', fontsize=FONT_SIZE_MAIN, )
    ax3.tick_params(axis='both', which='major', labelsize=FONT_SIZE_MAIN)
    ax3.tick_params(axis='both', which='minor', labelsize=FONT_SIZE_MAIN)
    
    # Determinar límite superior para distribución
    max_dist = np.max(frequency_distribution_smooth) * 1.2
    ax3.set_ylim(-1, max_dist)
    
    # Plotear curva de saturación acumulada
    ax1.semilogx(bin_centers_nm_sorted, cumulative_saturation, 
                 'b-', linewidth=2.5, label='$P_c$')
    
    # Plotear distribución de frecuencia
    ax3.semilogx(bin_centers_nm_sorted, frequency_distribution_smooth, 
                 'b--', linewidth=2.5, alpha=0.7, label='PSD')
    
    # Leyenda para PSD
    ax3.legend(loc='upper right', fontsize=FONT_SIZE_LEGEND)
    
    # Calcular y mostrar radios característicos si está habilitado
    if show_characteristic_radii and len(bin_centers_nm_sorted) > 3:
        
        # 1. Radio Apex - basado en la relación saturación/presión
        r_apex_nm, sat_apex = calculate_r_apex_from_distribution(
            bin_centers_nm_sorted, cumulative_saturation, gamma, theta
        )
        
        # 2. Radio Crítico - basado en el pico máximo de la distribución
        r_critical_nm, sat_critical = calculate_r_critical_from_distribution(
            bin_centers_nm_sorted, cumulative_saturation, frequency_distribution_smooth
        )
        
        # 3. Radio Promedio Geométrico Ponderado
        r_weighted_mean_nm, sat_weighted_mean = calculate_weighted_geometric_mean_radius(
            bin_centers_nm_sorted, cumulative_saturation
        )
        
        # Plotear los radios característicos si se calcularon exitosamente
        characteristic_radii = []
        
        # Obtener los límites actuales del eje Y para ajustar las líneas verticales
        y_min, y_max = ax1.get_ylim()
        y_line_start = y_min + (y_max - y_min) * 0.02  # Empezar la línea un 2% arriba del borde inferior
        
        if r_apex_nm is not None:
            # Línea vertical que no llega hasta el borde inferior
           # ax1.vlines(x=r_apex_nm, ymin=y_line_start, ymax=y_max, 
            #          colors='red', linestyles=':', linewidth=1.5, alpha=0.7)
            ax1.plot(r_apex_nm, sat_apex, 'ro', markersize=10, 
                    label=f'$R_{{apex}}$: {r_apex_nm:.2f} μm')
            characteristic_radii.append(('R_apex', r_apex_nm, sat_apex))
        
        if r_critical_nm is not None:
            #ax1.vlines(x=r_critical_nm, ymin=y_line_start, ymax=y_max, 
             #         colors='green', linestyles=':', linewidth=1.5, alpha=0.7)
            ax1.plot(r_critical_nm, sat_critical, 'g^', markersize=10, 
                    label=f'$R_c$: {r_critical_nm:.2f} μm')
            characteristic_radii.append(('R_critical', r_critical_nm, sat_critical))
        
        if r_weighted_mean_nm is not None:
            #ax1.vlines(x=r_weighted_mean_nm, ymin=y_line_start, ymax=y_max, 
             #         colors='purple', linestyles=':', linewidth=1.5, alpha=0.7)
            ax1.plot(r_weighted_mean_nm, sat_weighted_mean, 'ms', markersize=10, 
                    label=f'$R_{{WGM}}$: {r_weighted_mean_nm:.2f} μm')
            characteristic_radii.append(('R_weighted_mean', r_weighted_mean_nm, sat_weighted_mean))
        
        # Añadir leyenda si hay radios característicos
        if characteristic_radii:
            ax1.legend(loc='upper left', fontsize=FONT_SIZE_LEGEND)
    
    # Añadir título si se desea
    # ax1.set_title('Jump Size Distribution Analysis', fontsize=FONT_SIZE_TITLE, weight='bold', pad=20)
    
    # Ajustar layout
    plt.tight_layout()
    
    # Guardar figura  ################################################################
    output_filename = 'jump_analysis_MICP_format_with_radii.png' if show_characteristic_radii else 'jump_analysis_MICP_format.png'
    fig.savefig(output_filename, dpi=300, bbox_inches='tight', 
                facecolor='white', edgecolor='none')
    print(f"Gráfica guardada como: {output_filename}")
    
    # Mostrar si está en modo interactivo
    if plt.isinteractive():
        plt.show()
    
    # Imprimir estadísticas resumidas
    print("\n=== ESTADÍSTICAS DE ANÁLISIS DE SALTOS ===")
    print(f"Número total de saltos detectados: {sum(counts)}")
    print(f"Saturación total representada: {cumulative_saturation[-1]:.2f}%")
    
    # Encontrar el rango de radio con mayor contribución
    max_contrib_idx = np.argmax(total_jumps)
    max_contrib_bin = bin_stats[max_contrib_idx]
    
    print(f"\nRango de radio con mayor contribución:")
    print(f"  Rango: {max_contrib_bin['bin_range'][0]:.3f}-{max_contrib_bin['bin_range'][1]:.3f} μm")
    print(f"  Radio central: {max_contrib_bin['bin_center']:.3f} μm")
    print(f"  Contribución: {max_contrib_bin['total_jump']*100:.2f}% de la saturación")
    print(f"  Número de saltos: {max_contrib_bin['count']}")
    
    # Imprimir radios característicos si se calcularon
    if show_characteristic_radii and 'characteristic_radii' in locals():
        print("\nRadios característicos calculados:")
        for name, radius, saturation in characteristic_radii:
            print(f"  {name}: {radius:.2f} μm (Saturación: {saturation:.1f}%)")
    
    # Calcular radios característicos de la distribución acumulada
    # Interpolar para encontrar R25, R50, R75
    if len(cumulative_saturation) > 3:
        # Invertir para interpolar saturación -> radio
        # Necesitamos asegurarnos de que los valores estén ordenados para la interpolación
        unique_sats, unique_indices = np.unique(cumulative_saturation, return_index=True)
        unique_radii = bin_centers_nm_sorted[unique_indices]
        
        if len(unique_sats) > 1:
            f_interp = interpolate.interp1d(unique_sats, unique_radii, 
                                           kind='linear', fill_value='extrapolate')
            
            print(f"\nRadios característicos (basados en saturación acumulada):")
            for perc in [25, 50, 75]:
                if perc <= cumulative_saturation[-1]:
                    r_um = f_interp(perc)
                    print(f"  R{perc}: {r_um:.2f} μm")
    
    plt.close(fig)
    
    return fig


def calculate_r_apex_from_distribution(radii_nm, saturation, gamma, theta):
    """
    Calcula el radio apex usando el método de saturación/presión vs saturación
    adaptado para datos de distribución de saltos.
    """
    try:
        # Filtrar datos con saturación mínima del 10%
        sat_min = 10.0
        mask = saturation >= sat_min
        
        if np.sum(mask) < 3:  # Necesitamos al menos 3 puntos
            return None, None
        
        radii_filtered = radii_nm[mask]
        sat_filtered = saturation[mask]
        
        # Calcular presión para cada radio usando Washburn
        theta_rad = np.radians(theta)
        # Convertir μm a m para el cálculo
        pressure_psi = np.abs(2 * gamma * np.cos(theta_rad) / (radii_filtered * 1e-6)) * 0.000145038
        
        # Calcular ratio saturación/presión
        sat_pressure_ratio = sat_filtered / pressure_psi
        
        # Encontrar el máximo
        idx_max = np.argmax(sat_pressure_ratio)
        r_apex_nm = radii_filtered[idx_max]
        sat_apex = sat_filtered[idx_max]
        
        return r_apex_nm, sat_apex
        
    except Exception as e:
        print(f"Error calculando radio apex: {e}")
        return None, None


def calculate_r_critical_from_distribution(radii_nm, saturation, distribution):
    """
    Calcula el radio crítico basado en el pico máximo de la distribución dS/dlog(r).
    """
    try:
        # Encontrar el índice del máximo en la distribución
        idx_max = np.argmax(distribution)
        
        # Si el máximo está en los extremos, buscar picos internos
        if idx_max == 0 or idx_max == len(distribution) - 1:
            # Usar detección de picos
            peaks, properties = find_peaks(distribution, 
                                         height=np.max(distribution) * 0.15,
                                         distance=1)
            if len(peaks) > 0:
                # Tomar el pico con mayor altura
                idx_max = peaks[np.argmax(properties['peak_heights'])]
        
        r_critical_nm = radii_nm[idx_max]
        sat_critical = saturation[idx_max]
        
        return r_critical_nm, sat_critical
        
    except Exception as e:
        print(f"Error calculando radio crítico: {e}")
        return None, None


def calculate_weighted_geometric_mean_radius(radii_nm, saturation):
    """
    Calcula el radio promedio geométrico ponderado por los incrementos de saturación.
    """
    try:
        # Asegurar orden ascendente por radio para el cálculo correcto
        sorted_indices = np.argsort(radii_nm)
        radii_sorted = radii_nm[sorted_indices]
        sat_sorted = saturation[sorted_indices]
        
        # Calcular incrementos de saturación
        delta_sat = np.diff(sat_sorted)
        delta_sat = np.append(delta_sat, 0)  # Añadir 0 para el último punto
        
        # Usar valores absolutos y normalizar
        delta_sat_abs = np.abs(delta_sat)
        total_delta = np.sum(delta_sat_abs)
        
        if total_delta == 0:
            return None, None
        
        weights = delta_sat_abs / total_delta
        
        # Calcular media geométrica ponderada
        # log(r_mean) = sum(weight_i * log(r_i))
        log_r_mean = np.sum(weights * np.log(radii_sorted))
        r_mean_nm = np.exp(log_r_mean)
        
        # Interpolar para encontrar la saturación correspondiente
        # Necesitamos orden descendente de radio para la interpolación correcta
        f_interp = interpolate.interp1d(radii_nm, saturation, 
                                       kind='linear', fill_value='extrapolate')
        sat_mean = float(f_interp(r_mean_nm))
        
        return r_mean_nm, sat_mean
        
    except Exception as e:
        print(f"Error calculando radio promedio geométrico ponderado: {e}")
        return None, None

In [6]:

def create_animated_invasion_pc_3d_corrected(invasion_results, pc_analysis, fps=2, max_frames=30):
    """
    Versión corregida con mapeo consistente de coordenadas y formato homogeneizado de fuentes.
    CORREGIDO: Escalas en micrómetros para visualización 3D.
    """
    
    # ============= CONFIGURACIÓN GLOBAL DE ESTILO =============
    # Definir tamaños de fuente consistentes
    FONT_SIZE_MAIN = 20        # Tamaño principal para ejes y etiquetas
    FONT_SIZE_TITLE = 22       # Tamaño para títulos de subplots
    FONT_SIZE_SUPTITLE = 24    # Tamaño para título principal
    FONT_SIZE_SMALL = 16       # Tamaño para anotaciones y textos secundarios
    FONT_SIZE_LEGEND = 18      # Tamaño para leyendas
    
    # Configurar matplotlib para usar sans-serif
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.serif': ['Sans-Serif'], #'Times New Roman', 'DejaVu Serif', 'Times', 'serif'],
        'font.size': FONT_SIZE_MAIN,
        'axes.labelsize': FONT_SIZE_MAIN,
        'axes.titlesize': FONT_SIZE_TITLE,
        'xtick.labelsize': FONT_SIZE_MAIN,
        'ytick.labelsize': FONT_SIZE_MAIN,
        'legend.fontsize': FONT_SIZE_LEGEND,
        'figure.titlesize': FONT_SIZE_SUPTITLE,
    })
    # =========================================================
    
    print("\nCreando animación 3D de invasión y presión capilar...")
    
    medium = invasion_results['medium_connected']
    inv_satn = invasion_results['inv_saturation']
    inv_seq = invasion_results['inv_sequence']
    voxel_size = invasion_results['medium_info']['voxel_size_um']
    
    # Obtener parámetros del dominio
    domain_size_um = invasion_results['medium_info'].get('domain_size_um', None)
    
    # Si no está disponible domain_size_um, calcularlo
    if domain_size_um is None:
        shape = invasion_results['medium_info'].get('shape', medium.shape)
        domain_size_um = [dim * voxel_size for dim in shape]
    
    print(f"Dominio para animación: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³")
    
    step = 3
    scale_factor = voxel_size * step  # Factor de escala para convertir a μm
    medium_vis = medium[::step, ::step, ::step]
    inv_satn_vis = inv_satn[::step, ::step, ::step]
    inv_seq_vis = inv_seq[::step, ::step, ::step]
    
    print("Pre-calculando isosuperficie del medio...")
    verts_medium, faces_medium, _, _ = measure.marching_cubes(medium_vis.astype(float), level=0.5)
    # CORRECCIÓN: Convertir vértices a micrómetros y reordenar
    verts_medium_um = verts_medium * scale_factor
    verts_medium_reordered = verts_medium_um[:, [2, 1, 0]]
    
    unique_satns = np.unique(inv_satn[inv_satn > 0])
    
    if len(unique_satns) > max_frames:
        indices = np.linspace(0, len(unique_satns)-1, max_frames, dtype=int)
        frame_saturations = unique_satns[indices]
    else:
        frame_saturations = unique_satns
    
    print(f"Generando {len(frame_saturations)} frames...")
    
    surface_tension = 0.078
    contact_angle = 180
    cos_theta = np.cos(np.radians(contact_angle))
    dt = distance_transform_edt(medium)
    
    pc_data_frames = []
    
    for target_satn in frame_saturations:
        mask = (inv_satn <= target_satn) & (inv_satn > 0)
        if np.any(mask):
            max_seq = np.max(inv_seq[mask])
            
            sats_temp = []
            pres_temp = []
            
            seq_values = np.unique(inv_seq[(inv_seq > 0) & (inv_seq <= max_seq)])
            step_seq = max(1, len(seq_values) // 50)
            seq_values = seq_values[::step_seq]
            
            for seq_val in seq_values:
                mask_temp = (inv_seq <= seq_val) & (inv_seq > 0)
                sat = np.sum(mask_temp) / np.sum(medium)
                
                new_invaded = inv_seq == seq_val
                if np.any(new_invaded):
                    radius_vox = np.max(dt[new_invaded])
                    radius_um = radius_vox * voxel_size
                    
                    if radius_um > 0:
                        pressure_pa = -2 * surface_tension * cos_theta / (radius_um * 1e-6)
                        pressure_psi = pressure_pa * 1.45038e-4
                        
                        if 0 < pressure_psi < np.inf:
                            sats_temp.append(sat)
                            pres_temp.append(pressure_psi)
            
            pc_data_frames.append({
                'saturations': np.array(sats_temp),
                'pressures': np.array(pres_temp),
                'current_saturation': target_satn
            })
    
    # Aumentar tamaño de figura
    fig = plt.figure(figsize=(18, 9), facecolor='white')
    gs = fig.add_gridspec(1, 2, width_ratios=[1, 1], wspace=0.15)
    
    ax1 = fig.add_subplot(gs[0, 0], projection='3d')
    ax2 = fig.add_subplot(gs[0, 1])
    
    all_pressures = np.concatenate([frame['pressures'] for frame in pc_data_frames if len(frame['pressures']) > 0])
    if len(all_pressures) > 0:
        pres_min = np.percentile(all_pressures, 1)
        pres_max = np.percentile(all_pressures, 99)
    else:
        pres_min, pres_max = 10, 1000
    
    # Vista consistente
    ax1.view_init(elev=30, azim=45)
    
    def update(frame_idx):
        ax1.clear()
        ax2.clear()
        
        target_satn = frame_saturations[frame_idx]
        
        # Medio poroso con vértices reordenados en μm
        ax1.plot_trisurf(verts_medium_reordered[:, 0], verts_medium_reordered[:, 1], verts_medium_reordered[:, 2],
                        triangles=faces_medium, color='lightblue', alpha=0.15, 
                        edgecolor='none', linewidth=0)
        
        invasion_mask = (inv_satn_vis <= target_satn) & (inv_satn_vis > 0)
        
        if np.sum(invasion_mask) > 10:
            try:
                verts_inv, faces_inv, _, _ = measure.marching_cubes(
                    invasion_mask.astype(float), level=0.5
                )
                
                # CORRECCIÓN: Convertir vértices de invasión a micrómetros y reordenar
                verts_inv_um = verts_inv * scale_factor
                verts_inv_reordered = verts_inv_um[:, [2, 1, 0]]
                
                centroids = np.mean(verts_inv[faces_inv], axis=1)  # Usar índices originales para centroides
                colors = np.zeros((len(faces_inv), 3))
                
                progress_colors = centroids[:, 0] / medium_vis.shape[2]
                colors[:, 0] = 0.25 + 0.15 * progress_colors
                colors[:, 1] = 0.25 + 0.15 * progress_colors
                colors[:, 2] = 0.3 + 0.15 * progress_colors
                
                from mpl_toolkits.mplot3d.art3d import Poly3DCollection
                tri_collection = Poly3DCollection(verts_inv_reordered[faces_inv], 
                                                 facecolors=colors, 
                                                 alpha=0.9,
                                                 edgecolor='none',
                                                 linewidth=0)
                ax1.add_collection3d(tri_collection)
                
            except Exception as e:
                pass
        
        # CORRECCIÓN: Configurar ejes 3D con límites en micrómetros
        ax1.set_xlim([0, domain_size_um[2]])  # X en μm
        ax1.set_ylim([0, domain_size_um[1]])  # Y en μm
        ax1.set_zlim([0, domain_size_um[0]])  # Z en μm
        ax1.set_xlabel('X (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        ax1.set_ylabel('Y (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        ax1.set_zlabel('Z (μm)', fontsize=FONT_SIZE_MAIN,
                      rotation=0, ha='right', labelpad=10)
        
        # Configurar tamaño de ticks
        ax1.tick_params(axis='x', labelsize=FONT_SIZE_SMALL)
        ax1.tick_params(axis='y', labelsize=FONT_SIZE_SMALL)
        ax1.tick_params(axis='z', labelsize=FONT_SIZE_SMALL)
        
        # Configurar número de ticks para evitar amontonamiento
        ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
        ax1.yaxis.set_major_locator(plt.MaxNLocator(5))
        ax1.zaxis.set_major_locator(plt.MaxNLocator(5))
        
        ax1.set_title(f'3D\nSaturación: {target_satn:.3f}', 
                     fontsize=FONT_SIZE_TITLE)
        
        # Escala de referencia
        scale_length_um = domain_size_um[2] / 4
        ax1.text2D(0.05, 0.05, f'Escala: {scale_length_um:.1f} μm', 
                  transform=ax1.transAxes, fontsize=FONT_SIZE_SMALL,
                  bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
        
        # Mantener vista consistente
        ax1.view_init(elev=30, azim=45)
        
        # Panel 2: Curva PC
        pc_frame = pc_data_frames[frame_idx]
        
        if len(pc_frame['saturations']) > 0:
            ax2.semilogx(pc_frame['pressures'], pc_frame['saturations'], 
                        'g-', linewidth=2.5, alpha=0.8, label='PC dinámica')
            
            ax2.plot(pc_frame['pressures'][-1], pc_frame['saturations'][-1], 
                    'ro', markersize=12, zorder=5, label=f'Estado actual')
            
            if len(pc_frame['pressures']) > 5:
                peaks, properties = find_peaks(pc_frame['pressures'], 
                                             prominence=0.1*np.std(pc_frame['pressures']))
                if len(peaks) > 0:
                    ax2.plot(pc_frame['pressures'][peaks], pc_frame['saturations'][peaks],
                            'b^', markersize=12, label=f'Escalones ({len(peaks)})', zorder=4)
        
        ax2.set_ylim([0, 1])
        ax2.set_xlim([pres_min * 0.8, pres_max * 1.2])
        ax2.set_ylabel('Saturación', fontsize=FONT_SIZE_MAIN)
        ax2.set_xlabel('Presión (psi)', fontsize=FONT_SIZE_MAIN)
        ax2.set_title(f'Curva de Presión Capilar Dinámica\nFrame {frame_idx+1}/{len(frame_saturations)}', 
                     fontsize=FONT_SIZE_TITLE)
        ax2.grid(True, alpha=0.3, which='both')
        ax2.tick_params(axis='both', which='major', labelsize=FONT_SIZE_MAIN)
        ax2.legend(loc='upper right', fontsize=FONT_SIZE_LEGEND)
        
        if len(pc_frame['pressures']) > 0:
            current_pressure = pc_frame['pressures'][-1]
            radius_um = -2 * surface_tension * cos_theta / (current_pressure / 1.45038e-4 * 1e-6)
            ax2.text(0.05, 0.95, f'Radio: {radius_um:.3f} μm\nPresión: {current_pressure:.0f} psi', 
                    transform=ax2.transAxes, fontsize=FONT_SIZE_SMALL,
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
                    verticalalignment='top')
        
        progress = (frame_idx / (len(frame_saturations)-1)) * 100 if len(frame_saturations) > 1 else 0
        domain_info = f" | Domain: {domain_size_um[0]:.1f}×{domain_size_um[1]:.1f}×{domain_size_um[2]:.1f} μm³"
        plt.suptitle(f'Evolución 3D de Invasión y Presión Capilar - Progreso: {progress:.0f}%{domain_info}', 
                    fontsize=FONT_SIZE_SUPTITLE)
        
        plt.tight_layout(rect=[0, 0.03, 1, 1.05])
    
    print("Generando frames de animación...")
    anim = FuncAnimation(fig, update, frames=len(frame_saturations), 
                        interval=1000//fps, repeat=True)
    
    filename = 'invasion_pc_3d_evolution_corrected.gif'
    print(f"Guardando animación como {filename}...")
    anim.save(filename, writer=PillowWriter(fps=fps))
    print(f"Animación 3D guardada exitosamente")
    print(f"Dominio animado: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³")
    
    plt.close(fig)
    
    return anim



In [7]:


# ============= FUNCIONES AUXILIARES PARA RADIOS CARACTERÍSTICOS =============

def calculate_r_apex_from_distribution(radii_nm, saturation, gamma, theta):
    """
    Calcula el radio apex usando el método de saturación/presión vs saturación
    adaptado para datos de distribución de saltos.
    """
    try:
        # Filtrar datos con saturación mínima del 10%
        sat_min = 10.0
        mask = saturation >= sat_min
        
        if np.sum(mask) < 3:  # Necesitamos al menos 3 puntos
            return None, None
        
        radii_filtered = radii_nm[mask]
        sat_filtered = saturation[mask]
        
        # Calcular presión para cada radio usando Washburn
        theta_rad = np.radians(theta)
        # Convertir μm a m para el cálculo
        pressure_psi = np.abs(2 * gamma * np.cos(theta_rad) / (radii_filtered * 1e-6)) * 0.000145038
        
        # Calcular ratio saturación/presión
        sat_pressure_ratio = sat_filtered / pressure_psi
        
        # Encontrar el máximo
        idx_max = np.argmax(sat_pressure_ratio)
        r_apex_nm = radii_filtered[idx_max]
        sat_apex = sat_filtered[idx_max]
        
        return r_apex_nm, sat_apex
        
    except Exception as e:
        print(f"Error calculando radio apex: {e}")
        return None, None


def calculate_r_critical_from_distribution(radii_nm, saturation, distribution):
    """
    Calcula el radio crítico basado en el pico máximo de la distribución dS/dlog(r).
    """
    try:
        # Encontrar el índice del máximo en la distribución
        idx_max = np.argmax(distribution)
        
        # Si el máximo está en los extremos, buscar picos internos
        if idx_max == 0 or idx_max == len(distribution) - 1:
            # Usar detección de picos
            peaks, properties = find_peaks(distribution, 
                                         height=np.max(distribution) * 0.15,
                                         distance=1)
            if len(peaks) > 0:
                # Tomar el pico con mayor altura
                idx_max = peaks[np.argmax(properties['peak_heights'])]
        
        r_critical_nm = radii_nm[idx_max]
        sat_critical = saturation[idx_max]
        
        return r_critical_nm, sat_critical
        
    except Exception as e:
        print(f"Error calculando radio crítico: {e}")
        return None, None


def calculate_weighted_geometric_mean_radius(radii_nm, saturation):
    """
    Calcula el radio promedio geométrico ponderado por los incrementos de saturación.
    """
    try:
        # Asegurar orden ascendente por radio para el cálculo correcto
        sorted_indices = np.argsort(radii_nm)
        radii_sorted = radii_nm[sorted_indices]
        sat_sorted = saturation[sorted_indices]
        
        # Calcular incrementos de saturación
        delta_sat = np.diff(sat_sorted)
        delta_sat = np.append(delta_sat, 0)  # Añadir 0 para el último punto
        
        # Usar valores absolutos y normalizar
        delta_sat_abs = np.abs(delta_sat)
        total_delta = np.sum(delta_sat_abs)
        
        if total_delta == 0:
            return None, None
        
        weights = delta_sat_abs / total_delta
        
        # Calcular media geométrica ponderada
        # log(r_mean) = sum(weight_i * log(r_i))
        log_r_mean = np.sum(weights * np.log(radii_sorted))
        r_mean_nm = np.exp(log_r_mean)
        
        # Interpolar para encontrar la saturación correspondiente
        # Necesitamos orden descendente de radio para la interpolación correcta
        f_interp = interpolate.interp1d(radii_nm, saturation, 
                                       kind='linear', fill_value='extrapolate')
        sat_mean = float(f_interp(r_mean_nm))
        
        return r_mean_nm, sat_mean
        
    except Exception as e:
        print(f"Error calculando radio promedio geométrico ponderado: {e}")
        return None, None

# ============= FUNCIÓN PRINCIPAL UNIFICADA =============

def unified_pc_analysis_and_visualization(invasion_results, pc_analysis, 
                                         fps=2, max_frames=30, 
                                         save_static_frames=True,
                                         create_animation=True,
                                         analyze_jumps=True,
                                         show_characteristic_radii=True,
                                         min_jump_size=0.00001,
                                         n_bins=200):
    """
    Función unificada que integra:
    1. Cálculo de curva PC completa (radii_full, saturations_full, peaks1)
    2. Generación de frames 3D estáticos
    3. Creación de figura MICP con radios característicos
    4. Análisis detallado de saltos
    5. Visualización de distribución de saltos con radios característicos
    6. Creación de animación (opcional)
    
    Retorna un diccionario con todos los resultados y figuras generadas.
    """
    import matplotlib.ticker
    # ============= CONFIGURACIÓN GLOBAL DE ESTILO =============
    FONT_SIZE_MAIN = 20
    FONT_SIZE_TITLE = 22
    FONT_SIZE_SUPTITLE = 24
    FONT_SIZE_SMALL = 16
    FONT_SIZE_LEGEND = 14
    
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.serif': ['Sans-Serif'], #'Times New Roman', 'DejaVu Serif', 'Times', 'serif'],
        'font.size': FONT_SIZE_MAIN,
        'axes.labelsize': FONT_SIZE_MAIN,
        'axes.titlesize': FONT_SIZE_TITLE,
        'xtick.labelsize': FONT_SIZE_MAIN,
        'ytick.labelsize': FONT_SIZE_MAIN,
        'legend.fontsize': FONT_SIZE_LEGEND,
        'figure.titlesize': FONT_SIZE_SUPTITLE,
        'axes.linewidth': 1.5,
        'lines.linewidth': 2.5,
        'lines.markersize': 10,
        'figure.dpi': 300,
        'grid.alpha': 0.2,
        'grid.linestyle': '-'
    })
    
    print("\n" + "="*70)
    print("ANÁLISIS UNIFICADO DE PRESIÓN CAPILAR Y VISUALIZACIÓN")
    print("="*70)
    
    # ============= PARTE 1: EXTRACCIÓN DE DATOS =============
    print("\n[1/6] Extrayendo datos del medio poroso...")
    
    medium = invasion_results['medium_connected']
    inv_satn = invasion_results['inv_saturation']
    inv_seq = invasion_results['inv_sequence']
    voxel_size = invasion_results['medium_info']['voxel_size_um']
    
    domain_size_um = invasion_results['medium_info'].get('domain_size_um', None)
    if domain_size_um is None:
        shape = invasion_results['medium_info'].get('shape', medium.shape)
        domain_size_um = [dim * voxel_size for dim in shape]
    
    print(f"  Dominio: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³")
    
    # Parámetros físicos
    surface_tension = 0.0785
    contact_angle = 180
    cos_theta = np.cos(np.radians(contact_angle))
    
    # ============= PARTE 2: CÁLCULO DE CURVA PC COMPLETA =============
    print("\n[2/6] Calculando curva de presión capilar completa...")
    
    dt = distance_transform_edt(medium)
    
    saturations_full = []
    pressures_full = []
    radii_full = []
    sequence_values_full = []
    
    unique_sequences = np.unique(inv_seq[inv_seq > 0])
    step_seq = max(1, len(unique_sequences) // 500)
    seq_values = unique_sequences[::step_seq]
    
    for seq_val in seq_values:
        mask = (inv_seq <= seq_val) & (inv_seq > 0)
        sat = np.sum(mask) / np.sum(medium) * 100
        
        new_invaded = inv_seq == seq_val
        if np.any(new_invaded):
            radius_vox = np.max(dt[new_invaded])
            radius_um = radius_vox * voxel_size
            
            if radius_um > 0:
                pressure_pa = -2 * surface_tension * cos_theta / (radius_um * 1e-6)
                pressure_psi = pressure_pa * 1.45038e-4
                
                if 0 < pressure_psi < np.inf:
                    saturations_full.append(sat)
                    pressures_full.append(pressure_psi)
                    radii_full.append(radius_um)
                    sequence_values_full.append(seq_val)
    
    saturations_full = np.array(saturations_full)
    pressures_full = np.array(pressures_full)
    radii_full = np.array(radii_full)
    sequence_values_full = np.array(sequence_values_full)
    
    print(f"  Puntos calculados: {len(radii_full)}")
    
    # Identificar picos (pore-throats)
    peaks1, properties1 = find_peaks(pressures_full, 
                                    prominence=0.5*np.std(pressures_full),
                                    distance=1)
    print(f"  Pore-throats detectados: {len(peaks1)}")
    
    # Configurar límites para gráficas
    if len(pressures_full) > 0:
        pres_min = np.percentile(pressures_full, 1)
        pres_max = np.percentile(pressures_full, 99)
        rad_min = np.percentile(radii_full, 1)
        rad_max = np.percentile(radii_full, 99)
    else:
        pres_min, pres_max = 10, 1000
        rad_min, rad_max = 0.1, 100
    
    # ============= PARTE 3: FRAMES 3D ESTÁTICOS =============
    if save_static_frames:
        print("\n[3/6] Generando frames 3D estáticos...")
        
        # Preparar datos para visualización 3D
        step = 3
        scale_factor = voxel_size * step
        medium_vis = medium[::step, ::step, ::step]
        inv_satn_vis = inv_satn[::step, ::step, ::step]
        inv_seq_vis = inv_seq[::step, ::step, ::step]
        
        verts_medium, faces_medium, _, _ = measure.marching_cubes(medium_vis.astype(float), level=0.5)
        verts_medium_um = verts_medium * scale_factor
        
        unique_satns = np.unique(inv_satn[inv_satn > 0])
        if len(unique_satns) > max_frames:
            indices = np.linspace(0, len(unique_satns)-1, max_frames, dtype=int)
            frame_saturations = unique_satns[indices]
        else:
            frame_saturations = unique_satns
        
        # Función para crear frame 3D
        def create_3d_frame_only(frame_idx, target_saturation=None):
            fig = plt.figure(figsize=(12, 8), facecolor='white')
            ax = fig.add_subplot(111, projection='3d', position=[0.15, 0.1, 0.75, 0.85])
            
            ax.view_init(elev=30, azim=45)
            
            # Usar saturación específica o del frame
            if target_saturation is not None:
                target_satn = target_saturation
            else:
                target_satn = frame_saturations[frame_idx] if frame_idx < len(frame_saturations) else frame_saturations[0]
            
            # Medio poroso semi-transparente
            ax.plot_trisurf(verts_medium_um[:, 0], verts_medium_um[:, 1], verts_medium_um[:, 2],
                           triangles=faces_medium, color='lightblue', alpha=0.2, 
                           edgecolor='none', linewidth=0)
            
            # Invasión - CORREGIDO: mostrar invasión basada en target_satn
            if target_satn > 0:  # Cambio clave: si hay saturación, mostrar invasión
                invasion_mask = (inv_satn_vis <= target_satn) & (inv_satn_vis > 0)
                
                if np.sum(invasion_mask) > 10:
                    try:
                        verts_inv, faces_inv, _, _ = measure.marching_cubes(
                            invasion_mask.astype(float), level=0.5
                        )
                        verts_inv_um = verts_inv * scale_factor
                        
                        centroids = np.mean(verts_inv[faces_inv], axis=1)
                        colors = np.zeros((len(faces_inv), 3))
                        
                        # Gradiente de color basado en progreso espacial
                        progress_colors = centroids[:, 0] / medium_vis.shape[2]
                        colors[:, 0] = 0.25 + 0.15 * progress_colors
                        colors[:, 1] = 0.25 + 0.15 * progress_colors
                        colors[:, 2] = 0.3 + 0.15 * progress_colors
                        
                        tri_collection = Poly3DCollection(verts_inv_um[faces_inv], 
                                                         facecolors=colors, 
                                                         alpha=0.9,
                                                         edgecolor='none',
                                                         linewidth=0)
                        ax.add_collection3d(tri_collection)
                    except Exception as e:
                        print(f"    Error en marching cubes: {e}")
            
            # Configurar ejes
            ax.set_xlim([0, domain_size_um[2]])
            ax.set_ylim([0, domain_size_um[1]])
            ax.set_zlim([0, domain_size_um[0]])
            
            ax.set_xlabel('X (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
            ax.set_ylabel('Y (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
            ax.set_zlabel('Z (μm)', fontsize=FONT_SIZE_MAIN, rotation=0, labelpad=15)
            
            ax.tick_params(axis='x', labelsize=FONT_SIZE_SMALL)
            ax.tick_params(axis='y', labelsize=FONT_SIZE_SMALL)
            ax.tick_params(axis='z', labelsize=FONT_SIZE_SMALL)
            
            # Título con saturación correcta
            title = f'Saturation: {target_satn * 100:.1f}%'
            ax.set_title(title, fontsize=FONT_SIZE_TITLE, pad=10)
            
            ax.xaxis.pane.fill = False
            ax.yaxis.pane.fill = False
            ax.zaxis.pane.fill = False
            ax.grid(True, alpha=0.3)
            
            plt.subplots_adjust(left=0.0, right=1.0, bottom=0.0, top=0.95)
            
            return fig
        
        # Generar frames estáticos
        static_configs = [
            (0, 'Initial', 0.0),  # Sin invasión
            (int(len(frame_saturations) * 0.33), 'Intermediate_33%', frame_saturations[int(len(frame_saturations) * 0.33)]),
            (int(len(frame_saturations) * 0.66), 'Intermediate_66%', frame_saturations[int(len(frame_saturations) * 0.66)]),
            (len(frame_saturations) - 1, 'Final', frame_saturations[-1])
        ]
        
        for i, (idx, name, target_sat) in enumerate(static_configs):
            print(f"  Frame {i+1}/4: {name} (Saturación: {target_sat*100:.1f}%)")
            fig = create_3d_frame_only(idx, target_saturation=target_sat)
            filename = f'invasion_3d_frame_{i+1}_{name.lower()}.png'
            fig.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0.5, facecolor='white')
            plt.close(fig)
    
    # ============= PARTE 4: FIGURA MICP CON RADIOS CARACTERÍSTICOS =============
    print("\n[4/6] Generando figura de curva PC (formato MICP) con radios característicos...")
    
    # Primero calcular PSD para poder calcular radio crítico
    sorted_indices = np.argsort(radii_full)[::-1]
    radii_sorted_desc = radii_full[sorted_indices]
    sats_sorted_desc = saturations_full[sorted_indices]
    
    # Calcular diferencias de saturación
    dS = np.diff(sats_sorted_desc)
    
    # Calcular el ancho logarítmico de cada intervalo
    log_bin_widths = []
    for i in range(len(radii_sorted_desc)-1):
        dlog_r = np.abs(np.log10(radii_sorted_desc[i]) - np.log10(radii_sorted_desc[i+1]))
        if dlog_r == 0:
            dlog_r = 0.001
        log_bin_widths.append(dlog_r)
    
    log_bin_widths = np.array(log_bin_widths)
    
    # Calcular dS/dlog(r)
    frequency_distribution = np.abs(dS) / log_bin_widths
    
    # Puntos medios para graficar PSD
    radii_midpoints = []
    for i in range(len(radii_sorted_desc)-1):
        mid = np.sqrt(radii_sorted_desc[i] * radii_sorted_desc[i+1])
        radii_midpoints.append(mid)
    radii_midpoints = np.array(radii_midpoints)
    
    # Aplicar suavizado a PSD
    if len(frequency_distribution) > 5:
        window = min(5, len(frequency_distribution))
        if window % 2 == 0:
            window -= 1
        frequency_distribution_smooth = savgol_filter(frequency_distribution, 
                                                     window_length=window, 
                                                     polyorder=2)
        frequency_distribution_smooth = np.maximum(frequency_distribution_smooth, 0)
    else:
        frequency_distribution_smooth = frequency_distribution
    
    fig_pc = plt.figure(figsize=(8, 7), facecolor='white')
    ax = fig_pc.add_subplot(111)
    
    # Crear eje para PSD
    ax3 = ax.twinx()
    
    ax.grid(True, which="both", ls="-", alpha=0.2)
    
    # Configurar rangos
    log_margin = 0.2
    min_radius_plot = 10**(np.log10(rad_min) - log_margin)
    max_radius_plot = 10**(np.log10(rad_max) + log_margin)
    
    gamma = 0.078
    theta = 180
    theta_rad = np.radians(theta)
    pressure_min = np.abs(2 * gamma * np.cos(theta_rad) / (max_radius_plot * 1e-6)) * 0.000145038
    pressure_max = np.abs(2 * gamma * np.cos(theta_rad) / (min_radius_plot * 1e-6)) * 0.000145038
    
    # Eje X inferior (radio)
    ax.set_xscale('log')
    ax.set_xlim([max_radius_plot, min_radius_plot])
    ax.set_xlabel('Radio de garganta de poro (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
    ax.tick_params(axis='x', which='major', labelsize=FONT_SIZE_MAIN, pad=10)
    
    # Eje Y (saturación)
    ax.set_ylabel('Saturación (%)', fontsize=FONT_SIZE_MAIN, labelpad=10)
    ax.set_ylim([-1, 101])
    ax.yaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(5))
    ax.tick_params(axis='y', which='major', length=6, width=1.2, color='black')
    
    # Eje X superior (presión)
    ax2 = ax.twiny()
    ax2.set_xscale('log')
    ax2.set_xlim([pressure_min, pressure_max])
    ax2.set_xlabel('Presión (psi)', fontsize=FONT_SIZE_MAIN, labelpad=10)
    ax2.tick_params(axis='x', which='major', labelsize=FONT_SIZE_MAIN, pad=10)
    
    # Configurar eje Y derecho para PSD
    ax3.set_ylabel('dS/dlog(r) (%/log(μm))', fontsize=FONT_SIZE_MAIN, alpha=0)
            # Configurar los parámetros de los ticks
    ax3.tick_params(axis='y', which='major', labelsize=FONT_SIZE_MAIN, labelcolor='white', colors='white')
    ax3.tick_params(axis='y', which='minor', labelsize=FONT_SIZE_MAIN, labelcolor='white', colors='white')

    #ax3.tick_params(axis='y', which='major', labelsize=FONT_SIZE_MAIN, alpha=0)
    
    # Calcular límite superior para PSD
    if len(frequency_distribution_smooth) > 0:
        max_dist = np.max(frequency_distribution_smooth) * 1.2
    else:
        max_dist = 100
    ax3.set_ylim([-1, max_dist])
    
    # Graficar curva PC
    ax.plot(radii_full, saturations_full, 'b-', linewidth=2.5, alpha=0.8, label='$P_c$')
    
    # Graficar PSD
    #ax3.semilogx(radii_midpoints, frequency_distribution_smooth, 
    #            'b--', linewidth=2.5, alpha=0.7, label='PSD')
    
    # Marcar pore-throats
    if len(peaks1) > 0:
        ax.plot(radii_full[peaks1], saturations_full[peaks1],'r^', markersize=8, label=f'Gargantas de poro', zorder=3)
    
    # CALCULAR Y MOSTRAR RADIOS CARACTERÍSTICOS
    characteristic_radii = []
    
    if show_characteristic_radii and len(radii_full) > 3:
        
        # 1. Radio Apex
        r_apex_nm, sat_apex = calculate_r_apex_from_distribution(
            radii_full, saturations_full, gamma, theta,
        )
        
        # 2. Radio Crítico
     #   r_critical_nm, sat_critical = calculate_r_critical_from_distribution(
     #       radii_midpoints, saturations_full[:-1] if len(saturations_full) > len(radii_midpoints) else saturations_full[:len(radii_midpoints)], 
     #       frequency_distribution_smooth
     #   )
        
        # 3. Radio Promedio Geométrico Ponderado
     #   r_weighted_mean_nm, sat_weighted_mean = calculate_weighted_geometric_mean_radius(
     #       radii_full, saturations_full
     #   )
        
        # Plotear los radios característicos
       # if r_apex_nm is not None:
       #     ax.plot(r_apex_nm, sat_apex, 'ro', markersize=10, 
       #             label=f'$R_{{apex}}$: {r_apex_nm:.2f} μm', zorder=4)
       #     characteristic_radii.append(('R_apex', r_apex_nm, sat_apex))
        
       # if r_critical_nm is not None:
       #     ax.plot(r_critical_nm, sat_critical, 'g^', markersize=10, 
       #             label=f'$R_c$: {r_critical_nm:.2f} μm', zorder=4)
       #     characteristic_radii.append(('R_critical', r_critical_nm, sat_critical))
        
       # if r_weighted_mean_nm is not None:
       #     ax.plot(r_weighted_mean_nm, sat_weighted_mean, 'ms', markersize=10, 
       #             label=f'$R_{{WGM}}$: {r_weighted_mean_nm:.2f} μm', zorder=4)
       #     characteristic_radii.append(('R_weighted_mean', r_weighted_mean_nm, sat_weighted_mean))
    
    ax.legend(loc='upper left', fontsize=FONT_SIZE_LEGEND)
    #ax3.legend(loc='upper right', fontsize=FONT_SIZE_LEGEND)
    ##############################################################################################3  controlada por volumen
    plt.tight_layout()
    fig_pc.savefig('capillary_pressure_curve_micp_format.png', dpi=300, bbox_inches='tight', facecolor='white')
    
    # Imprimir radios característicos
    if characteristic_radii:
        print("\n  Radios característicos calculados:")
        for name, radius, saturation in characteristic_radii:
            print(f"    {name}: {radius:.2f} μm (Saturación: {saturation:.1f}%)")
    
    plt.close(fig_pc)
    
    # ============= PARTE 5: ANÁLISIS DE SALTOS =============
    jump_analysis_detailed = None
    fig_jumps = None
    
    if analyze_jumps:
        print("\n[5/6] Analizando distribución de saltos...")
        
        # Arrays para almacenar eventos
        all_radii = []
        all_jump_sizes = []
        all_saturations = []
        all_pressures = []
        
        unique_seq = np.unique(inv_seq[inv_seq > 0])
        prev_saturation = 0
        
        # Analizar cada paso
        for i, seq in enumerate(unique_seq):
            if i % 1000 == 0:
                print(f"  Procesando paso {i}/{len(unique_seq)}...")
            
            current_mask = (inv_seq <= seq) & (inv_seq > 0)
            current_saturation = np.sum(current_mask) / np.sum(medium)
            
            saturation_jump = current_saturation - prev_saturation
            
            if saturation_jump > min_jump_size:
                new_invaded = inv_seq == seq
                if np.any(new_invaded):
                    radius_vox = np.max(dt[new_invaded])
                    radius_um = radius_vox * voxel_size
                    
                    if radius_um > 0:
                        pressure_pa = -2 * surface_tension * cos_theta / (radius_um * 1e-6)
                        pressure_psi = pressure_pa * 1.45038e-4
                        
                        all_radii.append(radius_um)
                        all_jump_sizes.append(saturation_jump)
                        all_saturations.append(current_saturation)
                        all_pressures.append(pressure_psi)
            
            prev_saturation = current_saturation
        
        all_radii = np.array(all_radii)
        all_jump_sizes = np.array(all_jump_sizes)
        all_saturations = np.array(all_saturations)
        all_pressures = np.array(all_pressures)
        
        print(f"  Total de saltos detectados: {len(all_radii)}")
        
        # Análisis por bins
        if len(all_radii) > 0:
            r_min, r_max = all_radii.min() * 0.9, all_radii.max() * 1.1
            bins = np.logspace(np.log10(r_min), np.log10(r_max), n_bins)
            
            bin_stats = []
            for i in range(len(bins)-1):
                mask = (all_radii >= bins[i]) & (all_radii < bins[i+1])
                
                if np.any(mask):
                    stats = {
                        'bin_center': np.sqrt(bins[i] * bins[i+1]),
                        'bin_range': (bins[i], bins[i+1]),
                        'count': np.sum(mask),
                        'total_jump': np.sum(all_jump_sizes[mask]),
                        'mean_jump': np.mean(all_jump_sizes[mask]),
                        'std_jump': np.std(all_jump_sizes[mask]),
                        'max_jump': np.max(all_jump_sizes[mask]),
                        'radii': all_radii[mask],
                        'jumps': all_jump_sizes[mask],
                        'saturations': all_saturations[mask]
                    }
                else:
                    stats = {
                        'bin_center': np.sqrt(bins[i] * bins[i+1]),
                        'bin_range': (bins[i], bins[i+1]),
                        'count': 0,
                        'total_jump': 0,
                        'mean_jump': 0,
                        'std_jump': 0,
                        'max_jump': 0,
                        'radii': np.array([]),
                        'jumps': np.array([]),
                        'saturations': np.array([])
                    }
                
                bin_stats.append(stats)
            
            jump_analysis_detailed = {
                'all_radii': all_radii,
                'all_jump_sizes': all_jump_sizes,
                'all_saturations': all_saturations,
                'all_pressures': all_pressures,
                'bin_stats': bin_stats,
                'bins': bins
            }
            
            # Crear figura de distribución de saltos CON RADIOS CARACTERÍSTICOS
            print("  Generando visualización de distribución de saltos con radios característicos...")
            
            fig_jumps, ax1 = plt.subplots(figsize=(8, 7))
            
            ax3 = ax1.twinx()
            ax2 = ax1.twiny()
            
            # Preparar datos
            bin_centers = np.array([stat['bin_center'] for stat in bin_stats])
            counts = np.array([stat['count'] for stat in bin_stats])
            total_jumps = np.array([stat['total_jump'] for stat in bin_stats])
            
            sorted_indices = np.argsort(bin_centers)[::-1]
            bin_centers_sorted = bin_centers[sorted_indices]
            total_jumps_sorted = total_jumps[sorted_indices]
            
            cumulative_saturation = np.cumsum(total_jumps_sorted) * 100
            
            # Calcular PSD
            dS = total_jumps_sorted * 100
            log_bin_widths = []
            
            for i in range(len(bin_centers_sorted)):
                if i < len(bin_centers_sorted) - 1:
                    dlog_r = np.abs(np.log10(bin_centers_sorted[i]) - np.log10(bin_centers_sorted[i+1]))
                else:
                    if i > 0:
                        dlog_r = np.abs(np.log10(bin_centers_sorted[i-1]) - np.log10(bin_centers_sorted[i]))
                    else:
                        dlog_r = 0.1
                log_bin_widths.append(dlog_r)
            
            frequency_distribution_bins = dS / np.array(log_bin_widths)
            
            if len(frequency_distribution_bins) > 5:
                window = min(5, len(frequency_distribution_bins))
                if window % 2 == 0:
                    window -= 1
                frequency_distribution_bins_smooth = savgol_filter(frequency_distribution_bins, 
                                                             window_length=window, 
                                                             polyorder=2)
                frequency_distribution_bins_smooth = np.maximum(frequency_distribution_bins_smooth, 0)
            else:
                frequency_distribution_bins_smooth = frequency_distribution_bins
            
            # Configurar ejes
            ax1.grid(True, which="both", ls="-", alpha=0.2)
            
            min_radius_data = np.min(bin_centers_sorted)
            max_radius_data = np.max(bin_centers_sorted)
            
            log_margin = 0.2
            min_radius_plot = 10**(np.log10(min_radius_data) - log_margin)
            max_radius_plot = 10**(np.log10(max_radius_data) + log_margin)
            
            ax1.set_xscale('log')
            ax1.set_xlim(max_radius_plot, min_radius_plot)
            ax1.set_xlabel('Radio de garganta de poro (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
            ax1.set_ylabel('Saturación (%)', fontsize=FONT_SIZE_MAIN)
            ax1.set_ylim(-1, 101)
            ax1.yaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(5))
            
            pressure_min = np.abs(2 * gamma * np.cos(theta_rad) / (max_radius_plot * 1e-6)) * 0.000145038
            pressure_max = np.abs(2 * gamma * np.cos(theta_rad) / (min_radius_plot * 1e-6)) * 0.000145038
            
            ax2.set_xscale('log')
            ax2.set_xlim(pressure_min, pressure_max)
            ax2.set_xlabel('Presión (psi)', fontsize=FONT_SIZE_MAIN, labelpad=10)
            
            ax3.set_ylabel('dS/dlog(r) (%/log(μm))', fontsize=FONT_SIZE_MAIN, ) #######################################
            max_dist = np.max(frequency_distribution_bins_smooth) * 1.2
            ax3.set_ylim(-1, max_dist)
            
            # Graficar
            ax1.semilogx(bin_centers_sorted, cumulative_saturation, 
                        'b-', linewidth=2.5, label='$P_c$')
            ax3.semilogx(bin_centers_sorted, frequency_distribution_bins_smooth, 
                        'b--', linewidth=2.5, alpha=0.7, label='DTGP')
            
            # AÑADIR RADIOS CARACTERÍSTICOS AL ANÁLISIS DE SALTOS
            if show_characteristic_radii and len(bin_centers_sorted) > 3:
                
                # 1. Radio Apex
                r_apex_bins, sat_apex_bins = calculate_r_apex_from_distribution(
                    bin_centers_sorted, cumulative_saturation, gamma, theta
                )
                
                # 2. Radio Crítico
                r_critical_bins, sat_critical_bins = calculate_r_critical_from_distribution(
                    bin_centers_sorted, cumulative_saturation, frequency_distribution_bins_smooth
                )
                
                # 3. Radio Promedio Geométrico Ponderado
                r_weighted_mean_bins, sat_weighted_mean_bins = calculate_weighted_geometric_mean_radius(
                    bin_centers_sorted, cumulative_saturation
                )
                
                # Plotear los radios característicos
                if r_apex_bins is not None:
                    ax1.plot(r_apex_bins, sat_apex_bins, 'ro', markersize=10, 
                            label=f'$R_{{apex}}$: {r_apex_bins:.2f} μm', zorder=4)
                
                if r_critical_bins is not None:
                    ax1.plot(r_critical_bins, sat_critical_bins, 'g^', markersize=10, 
                            label=f'$R_c$: {r_critical_bins:.2f} μm', zorder=4)
                
                if r_weighted_mean_bins is not None:
                    ax1.plot(r_weighted_mean_bins, sat_weighted_mean_bins, 'ms', markersize=10, 
                            label=f'$R_{{WGM}}$: {r_weighted_mean_bins:.2f} μm', zorder=4)
            
            ax1.legend(loc='upper left', fontsize=FONT_SIZE_LEGEND)
            ax3.legend(loc='upper right', fontsize=FONT_SIZE_LEGEND)
            
            plt.tight_layout()
            
            # Determinar nombre del archivo
            output_filename = 'jump_analysis_MICP_format_with_radii.png' if show_characteristic_radii else 'jump_analysis_MICP_format.png'
            fig_jumps.savefig(output_filename, dpi=300, bbox_inches='tight', facecolor='white')
            
            # Imprimir estadísticas
            print("\n  === ESTADÍSTICAS DE ANÁLISIS DE SALTOS ===")
            print(f"  Número total de saltos detectados: {sum(counts)}")
            print(f"  Saturación total representada: {cumulative_saturation[-1]:.2f}%")
            
            # Encontrar el rango de radio con mayor contribución
            max_contrib_idx = np.argmax(total_jumps)
            max_contrib_bin = bin_stats[max_contrib_idx]
            
            print(f"\n  Rango de radio con mayor contribución:")
            print(f"    Rango: {max_contrib_bin['bin_range'][0]:.3f}-{max_contrib_bin['bin_range'][1]:.3f} μm")
            print(f"    Radio central: {max_contrib_bin['bin_center']:.3f} μm")
            print(f"    Contribución: {max_contrib_bin['total_jump']*100:.2f}% de la saturación")
            
            plt.close(fig_jumps)
    
    # ============= PARTE 6: ANIMACIÓN CORREGIDA =============
    anim = None
    if create_animation and save_static_frames:  # Solo si ya tenemos los datos preparados
        print("\n[6/6] Creando animación GIF mejorada...")
        
        # Crear figura para animación con ambos paneles
        fig_anim = plt.figure(figsize=(18, 9), facecolor='white')
        gs = fig_anim.add_gridspec(1, 2, width_ratios=[1, 1], wspace=0.15)
        ax1 = fig_anim.add_subplot(gs[0, 0], projection='3d')
        ax2 = fig_anim.add_subplot(gs[0, 1])
        
        # Picos para la animación (limitar a 20 más prominentes)
        peaks_anim = peaks1.copy()
        if len(peaks_anim) > 20:
            prominences = []
            for p in peaks_anim:
                if p > 0 and p < len(pressures_full) - 1:
                    prominence = pressures_full[p] - min(pressures_full[p-1], pressures_full[p+1])
                else:
                    prominence = 0
                prominences.append(prominence)
            prominences = np.array(prominences)
            sorted_indices = np.argsort(prominences)[::-1]
            peaks_anim = peaks_anim[sorted_indices[:20]]
            peaks_anim = np.sort(peaks_anim)
        
        # IMPORTANTE: Calcular secuencias correspondientes a las saturaciones
        # Esto es crítico para que la invasión progrese correctamente
        frame_sequences = []
        for target_satn in frame_saturations:
            # Encontrar la secuencia que corresponde a esta saturación
            current_sat = 0
            target_seq = 0
            for seq_val in np.unique(inv_seq_vis[inv_seq_vis > 0]):
                mask = (inv_seq_vis <= seq_val) & (inv_seq_vis > 0)
                current_sat = np.sum(mask) / np.sum(medium_vis)
                if current_sat >= target_satn:
                    target_seq = seq_val
                    break
            frame_sequences.append(target_seq)
        
        print(f"  Frames de animación: {len(frame_saturations)}")
        print(f"  Rango de saturación: {frame_saturations[0]*100:.1f}% - {frame_saturations[-1]*100:.1f}%")
        
        def update(frame_idx):
            ax1.clear()
            ax2.clear()
            
            target_satn = frame_saturations[frame_idx]
            target_seq = frame_sequences[frame_idx]
            
            # Panel 1: Vista isométrica 3D
            ax1.plot_trisurf(verts_medium_um[:, 0], verts_medium_um[:, 1], verts_medium_um[:, 2],
                            triangles=faces_medium, color='lightblue', alpha=0.15, 
                            edgecolor='none', linewidth=0)
            
            # INVASIÓN MEJORADA - Usar secuencia en lugar de saturación
            if target_seq > 0:
                # Usar inv_seq_vis para progresión más precisa
                invasion_mask = (inv_seq_vis <= target_seq) & (inv_seq_vis > 0)
                
                invaded_voxels = np.sum(invasion_mask)
                if invaded_voxels > 10:
                    try:
                        verts_inv, faces_inv, _, _ = measure.marching_cubes(
                            invasion_mask.astype(float), level=0.5
                        )
                        verts_inv_um = verts_inv * scale_factor
                        
                        # COLORES MÁS VIBRANTES
                        centroids = np.mean(verts_inv[faces_inv], axis=1)
                        colors = np.zeros((len(faces_inv), 3))
                        
                        # Esquema rojo-naranja (fuego/lava)
                        progress_x = centroids[:, 0] / medium_vis.shape[2]
                        colors[:, 0] = 0.5 + 0.5 * progress_x  # Rojo: 0.5 a 1.0
                        colors[:, 1] = 0.1 + 0.3 * progress_x  # Verde: 0.1 a 0.4
                        colors[:, 2] = 0.05 + 0.1 * (1 - progress_x)  # Azul: bajo
                        
                        tri_collection = Poly3DCollection(verts_inv_um[faces_inv], 
                                                         facecolors=colors, 
                                                         alpha=0.85,
                                                         edgecolor='none',
                                                         linewidth=0)
                        ax1.add_collection3d(tri_collection)
                    except Exception as e:
                        print(f"    Error en frame {frame_idx}: {e}")
            
            # Configurar ejes 3D
            ax1.set_xlim([0, domain_size_um[2]])
            ax1.set_ylim([0, domain_size_um[1]])
            ax1.set_zlim([0, domain_size_um[0]])
            
            ax1.set_xlabel('X (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
            ax1.set_ylabel('Y (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
            ax1.set_zlabel('Z (μm)', fontsize=FONT_SIZE_MAIN, rotation=0, ha='right', labelpad=10)
            
            ax1.tick_params(axis='x', labelsize=FONT_SIZE_SMALL)
            ax1.tick_params(axis='y', labelsize=FONT_SIZE_SMALL)
            ax1.tick_params(axis='z', labelsize=FONT_SIZE_SMALL)
            
            ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
            ax1.yaxis.set_major_locator(plt.MaxNLocator(5))
            ax1.zaxis.set_major_locator(plt.MaxNLocator(5))
            
            ax1.set_title(f'Saturación: {target_satn * 100:.1f}%', fontsize=FONT_SIZE_TITLE)
            ax1.view_init(elev=30, azim=45)
            
            # Panel 2: Curva PC
            ax2.set_xscale('log')
            ax2.set_xlim([rad_max * 1.2, rad_min * 0.8])
            ax2.set_ylim([-1, 101])
            ax2.set_xlabel('Radio de garganta de poro (μm)', fontsize=FONT_SIZE_MAIN)
            ax2.tick_params(axis='x', labelcolor='black', labelsize=FONT_SIZE_MAIN)
            ax2.set_ylabel('Saturación (%)', fontsize=FONT_SIZE_MAIN)
            ax2.tick_params(axis='y', labelsize=FONT_SIZE_MAIN)
            ax2.grid(True, alpha=0.3, which='both')
            
            # Eje superior (Presión)
            ax3 = ax2.twiny()
            ax3.set_xscale('log')
            ax3.set_xlim([pres_min * 0.8, pres_max * 1.2])
            ax3.set_xlabel('Presión (psi)', fontsize=FONT_SIZE_MAIN)
            ax3.tick_params(axis='x', labelcolor='black', labelsize=FONT_SIZE_MAIN)
            
            ax2.set_title('Presión capilar', fontsize=FONT_SIZE_TITLE, pad=40)
            
            # Curva completa
            ax2.plot(radii_full, saturations_full, 'lightgray', linewidth=2, alpha=0.5, label='$P_c$')
            
            # Índice actual
            current_idx = np.argmin(np.abs(saturations_full - target_satn * 100))
            
            # Porción recorrida con gradiente de color
            if current_idx > 0:
                # Usar color sólido verde para simplificar
                ax2.plot(radii_full[:current_idx+1], saturations_full[:current_idx+1], 
                        'g-', linewidth=3, alpha=0.8)
            
            # Marcador actual
            if current_idx < len(radii_full):
                current_radius = radii_full[current_idx]
                current_saturation = saturations_full[current_idx]
                
                ax2.plot(current_radius, current_saturation, 'ro', markersize=15, 
                        markerfacecolor='red', markeredgecolor='darkred', 
                        markeredgewidth=2, zorder=5)
                
                # Efecto de pulso
                pulse_size = 20 + 5 * np.sin(frame_idx * 0.5)
                ax2.plot(current_radius, current_saturation, 'ro', markersize=pulse_size, 
                        fillstyle='none', markeredgecolor='red', markeredgewidth=1, 
                        alpha=0.5, zorder=4)
            
            # Marcar picos
            if len(peaks_anim) > 0:
                ax2.plot(radii_full[peaks_anim], saturations_full[peaks_anim], 'b^', 
                        markersize=8, label='Gargantas de poro', zorder=3, alpha=0.6)
                
                passed_peaks = peaks_anim[peaks_anim <= current_idx]
                if len(passed_peaks) > 0:
                    ax2.plot(radii_full[passed_peaks], saturations_full[passed_peaks], 
                            'b^', markersize=14, zorder=4)
            
            ax2.legend(loc='upper left', fontsize=FONT_SIZE_LEGEND)
            
            # Título general
            progress = (frame_idx / (len(frame_saturations)-1)) * 100 if len(frame_saturations) > 1 else 0
            domain_info = f" | Dominio: {domain_size_um[0]:.1f}×{domain_size_um[1]:.1f}×{domain_size_um[2]:.1f} μm³"
            fig_anim.suptitle(f'Percolación de Invasión 3D Evolución - Progreso: {progress:.0f}%', 
                             fontsize=FONT_SIZE_SUPTITLE, y=1.08)
            
            plt.tight_layout(rect=[0, 0.03, 1, 1.15])
        
        # Crear animación
        anim = FuncAnimation(fig_anim, update, frames=len(frame_saturations), 
                            interval=1000//fps, repeat=True)
        
        filename = 'invasion_pc_3d_evolution.gif'
        print(f"  Guardando animación como {filename}...")
        print(f"  Esto puede tomar varios minutos...")
        anim.save(filename, writer=PillowWriter(fps=fps))
        print(f"  ✓ Animación guardada")
        
        plt.close(fig_anim)
    elif create_animation:
        print("\n[6/6] Animación omitida (requiere save_static_frames=True)")
        
    # ============= RETORNAR RESULTADOS =============
    print("\n" + "="*70)
    print("ANÁLISIS COMPLETADO")
    print("="*70)
    print("\nArchivos generados:")
    if save_static_frames:
        print("  • Frames 3D: 4 archivos PNG")
    print("  • Curva PC (MICP): capillary_pressure_curve_micp_format.png")
    if analyze_jumps and jump_analysis_detailed is not None:
        print("  • Análisis de saltos: " + output_filename)
    
    return {
        # Datos principales
        'radii_full': radii_full,
        'saturations_full': saturations_full,
        'pressures_full': pressures_full,
        'peaks1': peaks1,
        'sequence_values_full': sequence_values_full,
        
        # Radios característicos
        'characteristic_radii': characteristic_radii if 'characteristic_radii' in locals() else [],
        
        # Análisis de saltos
        'jump_analysis_detailed': jump_analysis_detailed,
        
        # Figuras
        'fig_pc': fig_pc if 'fig_pc' in locals() else None,
        'fig_jumps': fig_jumps if 'fig_jumps' in locals() else None,
        
        # Animación
        'animation': anim,
        
        # Parámetros
        'domain_size_um': domain_size_um,
        'voxel_size': voxel_size
    }

In [8]:
def main():
    """
    SUPER CÓDIGO 2000 CORREGIDO: Pipeline completo con cálculo correcto de EPC.
    """
    
    total_start = time.time()
    
    print("="*80)
    print("SUPER CÓDIGO 2000 CORREGIDO: ANÁLISIS COMPLETO CON EPC FIDEDIGNO")
    print("Versión corregida del cálculo de Euler-Poincaré")
    print("="*80)
    # PASO 1: Generar medio heterogéneo
    print("\nPASO 1: Generando medio poroso heterogéneo...")
    
    shape = (150, 150, 150)
    
    medium_info = create_heterogeneous_tight_rock(
        shape=shape,
        porosity_target=0.05,
        voxel_size_um=0.05,
        seed=42
    )
    
    # PASO 4: Simulación de invasión
    print("\nPASO 4: Ejecutando simulación de invasión...")
    
    invasion_results = run_invasion_fast(
        medium_info=medium_info,
        inlet_face='omnidirectional',
        max_iter=7500,
        inlet_thickness=3
        
    )
    
    if invasion_results is None:
        print("Error en simulación")
        return

    # PASO 5: Análisis de presión capilar
    print("\nPASO 5: Analizando presión capilar...")
    
    pc_analysis = analyze_capillary_pressure_fast(invasion_results) 
    
 
    # PASO 10: Animación 3D con curva PC
    
    print("\nPASO 10: Creando animación 3D de invasión y presión capilar...")
   

    results = unified_pc_analysis_and_visualization(
    invasion_results,
    pc_analysis,
    save_static_frames=True,     # Generar frames estáticos
    create_animation=True,        # También crear animación GIF
    analyze_jumps=True,          # Análisis de saltos
    show_characteristic_radii=True,  # Mostrar radios característicos
    fps=3,                       # Velocidad de animación
    max_frames=15                # Número de frames en la animación
)
    
        
        
        

    # RESUMEN FINAL mejorado
    total_time = time.time() - total_start
    
    print(f"\n{'='*80}")
    print(f"ANÁLISIS SUPER CÓDIGO 2000 CORREGIDO COMPLETADO")
    print(f"{'='*80}")
    print(f"Tiempo total: {total_time:.1f}s ({total_time/60:.1f} min)")
    
    print(f"\nArchivos generados:")
    print(f"• pc_connectivity_evolution_enhanced.png - Evolución con radios críticos")    

In [9]:
if __name__ == "__main__":
    # Ejecutar el análisis completo con EPC corregido
    main()

SUPER CÓDIGO 2000 CORREGIDO: ANÁLISIS COMPLETO CON EPC FIDEDIGNO
Versión corregida del cálculo de Euler-Poincaré

PASO 1: Generando medio poroso heterogéneo...
GENERANDO MEDIO POROSO HETEROGÉNEO OPTIMIZADO
Dimensiones: (150, 150, 150)
Porosidad objetivo: 5.0%
Tamaño de voxel: 0.050 μm

Distribución jerárquica de poros:
• Grandes (0.8 μm): 16 poros
• Medianos (0.4 μm): 101 poros
• Pequeños (0.2 μm): 506 poros

Creando red de canales base...
Porosidad base de canales: 0.011

Colocando poros jerárquicamente...
  Colocando 16 poros grandes...
  Colocando 101 poros medianos...
  Colocando 506 poros pequeños...

Porosidad después de colocación: 0.034
Ajustando porosidad...

Resultados finales:
• Porosidad final: 0.059
• Conectividad: 0.692
• Coeficiente de heterogeneidad: 0.585

PASO 4: Ejecutando simulación de invasión...

SIMULACIÓN DE INVASIÓN RÁPIDA
Dirección de invasión: omnidirectional
  Entrada omnidireccional (todas las caras)

Estadísticas del inlet:
  Voxels totales del inlet: 3890

  0%|          | 0/7499 [00:00<?, ?it/s]


Análisis de penetración por dirección:
  Penetración en X: 0.993
  Penetración en Y: 0.993
  Penetración en Z: 0.993
  Centro de masa (normalizado): (0.46, 0.48, 0.48)

RESUMEN DE RESULTADOS
Tiempo de simulación: 50.2s
Saturación máxima alcanzada: 0.909
Eficiencia de invasión: 0.909
Voxels invadidos: 135,585 / 149,194

PASO 5: Analizando presión capilar...

ANÁLISIS DE PRESIÓN CAPILAR
Analizando 232 puntos de 928 totales
Encontrados 64 picos en la curva PC
Escalones significativos en envolvente: 41

Radios críticos:
• Inflexión: 0.141 μm
• Percentil r35: 0.050 μm

PASO 10: Creando animación 3D de invasión y presión capilar...

ANÁLISIS UNIFICADO DE PRESIÓN CAPILAR Y VISUALIZACIÓN

[1/6] Extrayendo datos del medio poroso...
  Dominio: 7.5 × 7.5 × 7.5 μm³

[2/6] Calculando curva de presión capilar completa...
  Puntos calculados: 928
  Pore-throats detectados: 157

[3/6] Generando frames 3D estáticos...
  Frame 1/4: Initial (Saturación: 0.0%)
  Frame 2/4: Intermediate_33% (Saturación: 3